# Obtención detalla actividades (Web Scrapping)
(web Ayuntamiento de Madrid)

[Enlace portal web Ayuntamiento de Madrid Actividades Infantiles](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

Voy a trabajar como ejemplo con la [actividad 'Taller de ajedrez'](https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD)

<img src="../img/AyuntMadrid_actividades_01.png" width="500">

## Extracción actividad ejemplo de página 1

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Se encontraron {len(actividades)} actividades.")
        
        # Obtener la primera actividad para análisis detallado
        if actividades:
            primera_actividad = actividades[0]
            
            # Extraer el enlace de la primera actividad
            elemento_enlace = primera_actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_detalle = elemento_enlace['href']
                if not url_detalle.startswith('http'):
                    url_detalle = "https://www.madrid.es" + url_detalle
                
                print(f"Accediendo a la página de detalle: {url_detalle}")
                
                # Hacer una solicitud a la página de detalle
                response_detalle = requests.get(url_detalle)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"

                    # Extraer URL de imagen embebida
                    tramites_content = soup_detalle.find('div', class_='tramites-content')
                    if tramites_content:
                        imagen_elem = tramites_content.find('img')
                        if imagen_elem and imagen_elem.get('src'):
                            url_imagen = imagen_elem['src']
                            if not url_imagen.startswith('http'):
                                url_imagen = "https://www.madrid.es" + url_imagen
                            datos_actividad['url_imagen'] = url_imagen
                        else:
                            datos_actividad['url_imagen'] = "No disponible"
                    else:
                        datos_actividad['url_imagen'] = "No disponible"
                        
                    # Imprimir los resultados
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        print(f"{clave}: {valor}")
                    
                    # Crear DataFrame y guardar CSV
                    df = pd.DataFrame([datos_actividad])
                    directorio = '../data/raw/'
                    df.to_csv(f'{directorio}actividad_ejemplo.csv', index=False, encoding='utf-8-sig')
                    print(f"\nDatos guardados correctamente en '{directorio}actividad_ejemplo.csv'")
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("No se encontró el enlace a la página de detalle.")
        else:
            print("No se encontraron actividades.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Se encontraron 25 actividades.
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: 18:30 a 19:30 horas
fecha: Del miércoles 19 de marzo de 2025 al mi

In [2]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,18:30 a 19:30 horas,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...


## Extracción todas las actividades de una página

He comprobado la extracción para un elemento ejemplo. Voy a realizar un bucle para obtener dichos valores de cada una de las 25 actividades de esa pagina 1, y ponerlo en un dataframe para visualizarlo bien.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Crear directorio para guardar el CSV si no existe
directorio = '../data/raw/'

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
        print("Comenzando extracción de datos...\n")
        
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"Procesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_detalle = elemento_enlace['href']
                if not url_detalle.startswith('http'):
                    url_detalle = "https://www.madrid.es" + url_detalle
                
                print(f"  Accediendo a la página de detalle: {url_detalle}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(3)  # Esperar 3 segundos entre solicitudes
                response_detalle = requests.get(url_detalle)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Lista para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer edad, inscripción, periodicidad, día_días, horario
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar inscripción
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # Buscar periodicidad
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"
                        for patron in patrones_periodicidad:
                            if re.search(patron, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = patron
                                break
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                    
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                    
                    # Extraer URL de imagen embebida
                    tramites_content = soup_detalle.find('div', class_='tramites-content')
                    if tramites_content:
                        imagen_elem = tramites_content.find('img')
                        if imagen_elem and imagen_elem.get('src'):
                            url_imagen = imagen_elem['src']
                            if not url_imagen.startswith('http'):
                                url_imagen = "https://www.madrid.es" + url_imagen
                            datos_actividad['url_imagen'] = url_imagen
                        else:
                            datos_actividad['url_imagen'] = "No disponible"
                    else:
                        datos_actividad['url_imagen'] = "No disponible"

                        
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_detalle'] = url_detalle
                        
                    # Mostrar información básica de la actividad extraída
                    print(f"  ✓ Extraída: {datos_actividad['título']}")
                    print(f"    Lugar: {datos_actividad['lugar_nombre']}")
                    print(f"    Edad: {datos_actividad['edad']}")
                    print(f"    Fecha: {datos_actividad['fecha']}")
                    print(f"    Precio: {datos_actividad['precio']}")
                    print()
                    
                    # Añadir los datos de esta actividad a la lista general
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"  ✗ Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("  ✗ No se encontró el enlace a la página de detalle.")
        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df = pd.DataFrame(todas_actividades)
            nombre_archivo = f'{directorio}actividades_detalles_pagina1_{fecha_hoy}.csv'
            df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Consulta a fecha de 24-04-2025: se encontraron 25 actividades.
Comenzando extracción de datos...

Procesando actividad 1 de 25...
  Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD
  ✓ Extraída: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
    Lugar: Biblioteca Pública Municipal María Lejárraga (Hortaleza)
    Edad: de 6 a 17 años
    Fecha: Del miércoles 19 de marzo de 2025 al miércoles 4 de junio de 2025
    Precio: Gratuito

Procesando actividad 2 de 25...
  Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Manualidades/?vgnextfmt=default&vgnextoid=3da790ec79b45910

In [ ]:
# visualización idéntica al ejemplo
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la actividad de ejemplo
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Buscar el contenedor principal que contiene las actividades
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
        
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"\nProcesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_actividad = elemento_enlace['href']
                if not url_actividad.startswith('http'):
                    url_actividad = "https://www.madrid.es" + url_actividad
                
                print(f"Accediendo a la página de detalle: {url_actividad}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(2)  # Esperar 2 segundos entre solicitudes
                response_detalle = requests.get(url_actividad)
                
                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # diccionario para almacenar los datos de la actividad
                    datos_actividad = {}
                    
                    # Extraer título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"
                    
                    # Extraer descripción
                    descripcion_elem = soup_detalle.find('div', class_='tiny-text')
                    if descripcion_elem and descripcion_elem.p:
                        datos_actividad['descripción'] = descripcion_elem.p.text.strip()
                    else:
                        datos_actividad['descripción'] = "Sin descripción"
                    
                    # Extraer en el texto general tiny_text  (edad, inscripción, periodicidad, día_días, horario, otros datos)
                    tiny_text = soup_detalle.find('div', class_='tiny-text')
                    if tiny_text:
                        contenido_texto = tiny_text.get_text()
                        
                        # Buscar edad
                        patron_edad = re.search(r'(\d+\s+a\s+\d+\s+años|de\s+\d+\s+a\s+\d+\s+años)', contenido_texto)
                        datos_actividad['edad'] = patron_edad.group(0) if patron_edad else "No especificada"
                        
                        # Buscar edad - versión más amplia pero simplificada
                        datos_actividad['edad'] = "No especificada"
                        
                        # Buscar primero en los blockquotes
                        for blockquote in tiny_text.find_all('blockquote'):
                            if 'Edad:' in blockquote.text and 'años' in blockquote.text:
                                datos_actividad['edad'] = blockquote.text.strip()
                                break

                        # Si no se encontró en blockquotes, buscar en el contenido general
                        if datos_actividad['edad'] == "No especificada":
                            # Dividir el contenido en posibles frases usando puntuación
                            posibles_frases = re.split(r'[.,;:()\n]', contenido_texto)
                            for frase in posibles_frases:
                                frase_limpia = frase.strip()
                                if 'años' in frase_limpia and (
                                    'edad' in frase_limpia.lower() or 
                                    'niñ' in frase_limpia.lower() or 
                                    'de' in frase_limpia.lower() or 
                                    'a partir' in frase_limpia.lower()
                                ):
                                    datos_actividad['edad'] = frase_limpia
                                    break            
                        
                        # # Buscar inscripción
                        # patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        # datos_actividad['inscripción'] = patron_inscripcion.group(0).strip() if patron_inscripcion else "No especificada"
                        
                        # EXTRACCIÓN MEJORADA DE INSCRIPCIÓN
                        # Primero buscamos en el texto como antes
                        patron_inscripcion = re.search(r'([^.]*inscripción[^.]*\.)', contenido_texto, re.IGNORECASE)
                        texto_inscripcion = patron_inscripcion.group(0).strip() if patron_inscripcion else ""

                        # Buscar párrafo que contenga "Inscripción" como título fuerte o en encabezado
                        inscripcion_parrafo = ""
                        for p in tiny_text.find_all(['p']):
                            # Verificar si contiene la palabra "inscr" en cualquier parte del párrafo
                            if re.search(r'inscr', p.text, re.IGNORECASE):
                                inscripcion_parrafo = p.text.strip()
                                break

                        # Buscar en div.info-actividad si existe
                        info_actividad = soup_detalle.find('div', class_='info-actividad')
                        if info_actividad:
                            # Buscar el encabezado de inscripción y luego el párrafo de texto que le sigue
                            inscripcion_header = info_actividad.find('h4', class_='inscripcion')
                            if inscripcion_header and inscripcion_header.find_next('p', class_='text-date'):
                                texto_fecha_inscripcion = inscripcion_header.find_next('p', class_='text-date').text.strip()
                                if texto_fecha_inscripcion:
                                    inscripcion_parrafo = texto_fecha_inscripcion

                        # Asignar el valor final de inscripción según lo que se haya encontrado
                        if texto_inscripcion or inscripcion_parrafo:
                            datos_actividad['inscripción'] = texto_inscripcion
                            if inscripcion_parrafo:
                                if texto_inscripcion:
                                    datos_actividad['inscripción'] += ". " + inscripcion_parrafo
                                else:
                                    datos_actividad['inscripción'] = inscripcion_parrafo
                        else:
                            datos_actividad['inscripción'] = "No especificada"

                        # Periodicidad
                        datos_actividad['periodicidad'] = "No especificada"
                        # Agregar detección de patrones como "un martes al mes", "un jueves cada semana", etc.
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        periodos = {
                            'semana': 'semanal',
                            'mes': 'mensual',
                            'año': 'anual'
                        }

                        for dia in dias_semana:
                            for periodo_texto, periodicidad_valor in periodos.items():
                                if re.search(rf'un\s+{dia}\s+(al|cada)\s+{periodo_texto}', contenido_texto, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = periodicidad_valor
                                    break
                            if datos_actividad['periodicidad'] != "No especificada":
                                break
                            
                        # Buscar periodicidad explicitamente declarada                 
                        patrones_periodicidad = ['diaria', 'semanal', 'quincenal', 'mensual']
                        datos_actividad['periodicidad'] = "No especificada"

                        # Comprobar primero si hay días de la semana en plural (indicativo de periodicidad semanal)
                        dias_semana_plural = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábados', 'domingos']
                        for dia in dias_semana_plural:
                            # Buscar patrones como "Los martes", "Todos los jueves", etc.
                            if re.search(r'(los|las|todos\s+los|todas\s+las)\s+' + dia, contenido_texto, re.IGNORECASE):
                                datos_actividad['periodicidad'] = "semanal"
                                break

                        # Si no se encontró periodicidad con los días en plural, buscar con los patrones originales
                        if datos_actividad['periodicidad'] == "No especificada":
                            for patron in patrones_periodicidad:
                                if re.search(patron, contenido_texto, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = patron
                                    break
                        
                        
                        # Buscar días
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        dias_encontrados = []
                        for dia in dias_semana:
                            if re.search(dia, contenido_texto, re.IGNORECASE):
                                dias_encontrados.append(dia)
                        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"
                        
                        # Buscar horario
                        # patron_horario = re.search(r'(\d{1,2}[:.]\d{2}\s*a\s*\d{1,2}[:.]\d{2}\s*horas)', contenido_texto)
                        # datos_actividad['horario'] = patron_horario.group(0) if patron_horario else "No especificado"
                        
                        # Buscar horario con delimitación más precisa (rango u hora única)
                        datos_actividad['horario'] = "No especificado"

                        # Dividir el texto en frases usando signos de puntuación y saltos de línea
                        fragmentos = re.split(r'[.,;()\n]', contenido_texto)

                        for fragmento in fragmentos:
                            fragmento = fragmento.strip()
                            if 'horas' in fragmento.lower():
                                # Buscar rango de horas o una sola hora con "horas"
                                if re.search(r'\d{1,2}[:.]\d{2}\s*(a|-)\s*\d{1,2}[:.]\d{2}', fragmento) or \
                                re.search(r'(\d{1,2}[:.]\d{2})\s*horas', fragmento, re.IGNORECASE) or \
                                re.search(r'\d{1,2}\s*horas', fragmento, re.IGNORECASE):
                                    datos_actividad['horario'] = fragmento
                                    break
                                
                    # Ahora en la parte de info-actividad,buscamos:
                    # Extraer horario, si antes estuviera vacío
                    if datos_actividad['horario'] == "No especificado" and info_actividad:
                        fragmentos_info = re.split(r'[.,;()\n]', info_actividad.get_text())
                        for fragmento in fragmentos_info:
                            fragmento = fragmento.strip()
                            if 'horas' in fragmento.lower():
                                if re.search(r'\d{1,2}[:.]\d{2}\s*(a|-)\s*\d{1,2}[:.]\d{2}', fragmento) or \
                                re.search(r'\d{1,2}[:.]\d{2}\s*horas', fragmento, re.IGNORECASE) or \
                                re.search(r'\d{1,2}\s*horas', fragmento, re.IGNORECASE):
                                    datos_actividad['horario'] = fragmento
                                    break
                     
                    # Extraer fecha
                    fecha_elem = soup_detalle.find('p', class_='text-date')
                    datos_actividad['fecha'] = fecha_elem.text.strip() if fecha_elem else "Sin fecha"
                
                    
                    # Extraer lugar_nombre
                    lugar_elem = soup_detalle.find('a', class_='url fn')
                    datos_actividad['lugar_nombre'] = lugar_elem.text.strip() if lugar_elem else "Sin lugar"
                    
                    # Extraer lugar_dirección
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        direccion = re.sub(r'\s+', ' ', direccion)  # Eliminar espacios múltiples
                        datos_actividad['lugar_dirección'] = direccion
                    else:
                        datos_actividad['lugar_dirección'] = "Sin dirección"
                    
                    # Extraer precio y recomendación
                    actividades_info = soup_detalle.find('div', class_='actividades-info')
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        datos_actividad['precio'] = precio_elem.text.strip() if precio_elem else "No especificado"
                        
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        datos_actividad['recomendación'] = recomendacion_elem.text.strip() if recomendacion_elem else "No especificado"
                    else:
                        datos_actividad['precio'] = "No especificado"
                        datos_actividad['recomendación'] = "No especificado"

                    # Si no se encontró precio o es "No especificado", buscar en tiny-text si hay menciones a euros
                    if datos_actividad['precio'] == "No especificado":
                        tiny_text = soup_detalle.find('div', class_='tiny-text')
                        if tiny_text:
                            # Buscar todos los párrafos que contienen "euro" o "euros"
                            precio_parrafos = []
                            for p in tiny_text.find_all(['p', 'blockquote']):
                                if re.search(r'euro[s]?', p.text, re.IGNORECASE):
                                    precio_parrafos.append(p.text.strip())
                            
                            # Si se encontraron párrafos con precio, combinarlos
                            if precio_parrafos:
                                datos_actividad['precio'] = ". ".join(precio_parrafos)
                    
                    # Extraer url_ampliar_info (buscar bajo el encabezado "Amplíe información")
                    amplia_info_header = soup_detalle.find('h4', class_='title8', string='Amplíe información')
                    if amplia_info_header and amplia_info_header.find_next('p') and amplia_info_header.find_next('p').find('a'):
                        url_ampliar = amplia_info_header.find_next('p').find('a')['href']
                        if not url_ampliar.startswith('http'):
                            url_ampliar = "https://www.madrid.es" + url_ampliar
                        datos_actividad['url_ampliar_info'] = url_ampliar
                    else:
                        datos_actividad['url_ampliar_info'] = "No disponible"
                        
                    # Extraer URL de imagen embebida
                    tramites_content = soup_detalle.find('div', class_='tramites-content')
                    if tramites_content:
                        imagen_elem = tramites_content.find('img')
                        if imagen_elem and imagen_elem.get('src'):
                            url_imagen = imagen_elem['src']
                            if not url_imagen.startswith('http'):
                                url_imagen = "https://www.madrid.es" + url_imagen
                            datos_actividad['url_imagen'] = url_imagen
                        else:
                            datos_actividad['url_imagen'] = "No disponible"
                    else:
                        datos_actividad['url_imagen'] = "No disponible"

                        
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_actividad'] = url_actividad
                        
                    # Imprimir los resultados en el formato exacto solicitado
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                            print(f"{clave}: {valor}")
                    
                    # Añadir los datos de esta actividad a la lista general
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
            else:
                print("No se encontró el enlace a la página de detalle.")
        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df0 = pd.DataFrame(todas_actividades)
            
            # Definir directorio para guardar el CSV
            directorio = '../data/raw/' # Directorio de guardado de datos

            nombre_archivo = f'{directorio}actividades_detalles_pagina1_v0_{fecha_hoy}.csv'
            df0.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Contenedor principal encontrado.
Consulta a fecha de 24-04-2025: se encontraron 25 actividades.

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: Dirigidos a usuarios de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días:

In [6]:
df0

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_actividad
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,Dirigidos a usuarios de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,Las sesiones de este taller se celebrarán los ...,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
1,Manualidades,"Taller de manualidades ideado, coordinado y ej...",Dirigido a niñas y niños de 6 a 12 años,El proceso de inscripción finaliza el 23/06/2025,No especificada,martes,de 18 a 19:30 horas,El proceso de inscripción finaliza el 23/06/2025,Biblioteca Pública Municipal Gabriel García Má...,"PLAZA PUEBLO, 2 (CENTRO CULTURAL ORCASUR) 2804...",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
2,Talleres de Price (De 7 a 9 años),¡Diviértete aprendiendo mientras descubres tod...,Edad: De 7 a 9 años,No especificada,No especificada,No especificado,Horario: De 10 a 12 horas,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
3,Talleres del Price (5 y 6 años),¡Diviértete aprendiendo mientras descubres tod...,Edad: 5 y 6 años,No especificada,No especificada,No especificado,No especificado,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",Abono trimestral (10 sesiones): 140 euros\nAbo...,No especificado,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
4,LUDOTECAS,Sábados de abril y laborables no lectivos de S...,para niños y niñas con edad comprendidas entre...,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,21 y 26 de abril de 10:30 a 13:30 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Valverde (Fuencarral - El...,"PLAZA ISLAS AZORES, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
5,TALLERES INFANTILES,Sábados 5 y 19. MONSTRUOS DEL RIO: Para menore...,Para menores de 6 a 8 años,No especificada,No especificada,"sábado, domingo",No especificado,Del sábado 5 de abril de 2025 al domingo 27 de...,Centro de Interpretación de la Naturaleza Mont...,"Carretera M-607 Km 13 L-10, null MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
6,Taller infantil. Reciclo y utilizo,Reciclado creativo para pequeños ecologistas: ...,niños y niñas de 6 a 12 años descubrirán los t...,INSCRIPCIONES: en el centro cultural el primer...,semanal,sábado,de 10:30 a 12 horas,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Cultural Carril del Conde (Hortaleza),"CALLE CARRIL DEL CONDE, 57 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
7,LUDOTECAS,Sábados de abril y días laborales no lectivos ...,para niños y niñas con edades comprendidas ent...,"Actividad gratuita y de carácter abierto, sin ...",No especificada,sábado,No especificado,Del sábado 5 de abril de 2025 al sábado 26 de ...,Centro Sociocultural Alfonso XII (Fuencarral -...,"CALLE MIRA EL RIO, 4 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
8,Deportes populares y tradicionales para familias,Para todos los públicos. Sin inscripción previ...,No especificada,Sin inscripción previa.. Para todos los púb

In [28]:
# código original reorganizado y mejorado ChatGPT
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la primera pagina, en la que trabajaré primeramente
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Lista para almacenar los datos de todas las actividades
todas_actividades = []

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
       
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            print(f"\nProcesando actividad {indice} de {len(actividades)}...")
            
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_actividad = elemento_enlace['href']
                if not url_actividad.startswith('http'):
                    url_actividad = "https://www.madrid.es" + url_actividad
                
                print(f"Accediendo a la página de detalle: {url_actividad}")
                
                # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
                time.sleep(2)       # Esperar 2 segundos entre solicitudes
                response_detalle = requests.get(url_actividad)

                if response_detalle.status_code == 200:
                    soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
                    
                    # Diccionario para almacenar los datos de la actividad
                    datos_actividad = {}

                    contenedor_actividad = soup_detalle.find('div', class_='Panel 1.1')
                    if not contenedor_actividad:
                        print("No se encontró el contenedor de contenido de la actividad.")
                        continue

                    tramites_content = contenedor_actividad.find('div', class_='tramites-content')
                    tiny_text = contenedor_actividad.find('div', class_='tiny-text')
                    # tiny_text = soup_detalle.find('div', class_='tiny-text')  #antes, busca desde la raiz html y puede ser más lento y con errores
                    actividades_info = contenedor_actividad.find('div', class_='actividades-info')
                    info_actividad_fecha_lugar = contenedor_actividad.find('div', class_='info-actividad')

                    # Título
                    titulo_elem = soup_detalle.find('h3', class_='summary-title')
                    datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"

                    # Descripción
                    datos_actividad['descripción'] = tiny_text.p.text.strip() if tiny_text and tiny_text.p else "Sin descripción"

                    # Edad
                    datos_actividad['edad'] = "No especificada"

                    # Función auxiliar para extraer el rango de edad de un texto
                    def extraer_rango_edad(texto):
                        # Patrón para "de X a Y años"
                        patron_rango = re.search(r'de\s+(\d+)\s+a\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_rango:
                            return f"de {patron_rango.group(1)} a {patron_rango.group(2)} años"
                        
                        # Patrón para "Edad: X y Y años" o similares con dos puntos
                        if ':' in texto:
                            partes = texto.split(':', 1)
                            if 'años' in partes[1]:
                                # Extraer solo la parte que contiene la edad después de los dos puntos
                                edad_parte = partes[1].strip()
                                # Si hay texto adicional después de "años", cortarlo
                                if ' años' in edad_parte:
                                    edad_parte = edad_parte[:edad_parte.find(' años') + 5]
                                return edad_parte
                        
                        # Patrón para "entre X y Y años"
                        patron_entre = re.search(r'entre\s+(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_entre:
                            return f"entre {patron_entre.group(1)} y {patron_entre.group(2)} años"

                        # Patrón para "X y Y años" (nomralmente consecutivos)
                        patron_y = re.search(r'(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_y:
                            return f"{patron_y.group(1)} y {patron_y.group(2)} años"
                                                
                        # Patrón para "a partir de X años"
                        patron_a_partir = re.search(r'a\s+partir\s+de\s+(\d+)\s+años', texto, re.IGNORECASE)
                        if patron_a_partir:
                            return f"a partir de {patron_a_partir.group(1)} años"
                        
                        # Si contiene años y alguna palabra clave pero no encaja en patrones anteriores
                        if 'años' in texto and any(palabra in texto.lower() for palabra in ['edad', 'niñ', 'de', 'a partir']):
                            return texto.strip()
                        
                        return None

                    # Buscar en blockquote dentro de tiny_text
                    if tiny_text:
                        for blockquote in tiny_text.find_all('blockquote'):
                            if 'Edad:' in blockquote.text and 'años' in blockquote.text:
                                edad_extraida = extraer_rango_edad(blockquote.text)
                                if edad_extraida:
                                    datos_actividad['edad'] = edad_extraida
                                    break
                        
                        # Si aún no se ha encontrado, buscar en todo el texto de tiny_text
                        if datos_actividad['edad'] == "No especificada":
                            posibles_frases = re.split(r'[.,;:()\n]', tiny_text.get_text())
                            for frase in posibles_frases:
                                edad_extraida = extraer_rango_edad(frase)
                                if edad_extraida:
                                    datos_actividad['edad'] = edad_extraida
                                    break

                    # Buscar en tramites_content si aún no se ha encontrado
                    if datos_actividad['edad'] == "No especificada" and tramites_content:
                        posibles_frases = re.split(r'[.,;:()\n]', tramites_content.get_text())
                        for frase in posibles_frases:
                            edad_extraida = extraer_rango_edad(frase)
                            if edad_extraida:
                                datos_actividad['edad'] = edad_extraida
                                break

                    # Inscripción
                    texto_inscripcion = ""
                    inscripcion_parrafo = ""

                    # Primero buscar en div.info-actividad si existe:
                    if info_actividad_fecha_lugar:
                        h4 = info_actividad_fecha_lugar.find('h4', class_='inscripcion')
                        if h4:
                            siguiente_p = h4.find_next('p', class_='text-date')
                            if siguiente_p:
                                inscripcion_parrafo = siguiente_p.text.strip()

                    # Después buscar también  en tiny_text (independientemente de si ya encontramos algo)
                    if tiny_text:
                        match = re.search(r'([^.]*inscripción[^.]*\.)', tiny_text.get_text(), re.IGNORECASE)
                        if match:
                            texto_inscripcion = match.group(0).strip()
                        
                        # Además, buscar en párrafos específicos dentro de tiny_text
                        for p in tiny_text.find_all('p'):
                            # Verificar si contiene la palabra "inscr" en cualquier parte del párrafo
                            if re.search(r'inscr', p.text, re.IGNORECASE):
                                # Si ya tenemos contenido de info_actividad, concatenamos
                                if texto_inscripcion:
                                    texto_inscripcion += ". " + p.text.strip()
                                else:
                                    texto_inscripcion = p.text.strip()
                                break

                    # Asignar el valor final de inscripción según lo que se haya encontrado
                    if inscripcion_parrafo or texto_inscripcion:
                        # Si tenemos ambos contenidos, los concatenamos
                        if inscripcion_parrafo and texto_inscripcion:
                            datos_actividad['inscripción'] = inscripcion_parrafo + ". " + texto_inscripcion
                        elif inscripcion_parrafo:
                            datos_actividad['inscripción'] = inscripcion_parrafo
                        else:
                            datos_actividad['inscripción'] = texto_inscripcion
                    else:
                        datos_actividad['inscripción'] = "No especificada"

                    # Periodicidad
                    datos_actividad['periodicidad'] = "No especificada"
                    if tiny_text:
                        contenido = tiny_text.get_text()
                        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                        periodos = {'semana': 'semanal', 'mes': 'mensual', 'año': 'anual'}
                        for dia in dias_semana:
                            for periodo, valor in periodos.items():
                                if re.search(rf'un\s+{dia}\s+(al|cada)\s+{periodo}', contenido, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = valor
                                    break
                            if datos_actividad['periodicidad'] != "No especificada":
                                break
                        if datos_actividad['periodicidad'] == "No especificada":
                            patrones = ['diaria', 'semanal', 'quincenal', 'mensual']
                            for patron in patrones:
                                if re.search(patron, contenido, re.IGNORECASE):
                                    datos_actividad['periodicidad'] = patron
                                    break

                    # Día/días
                    datos_actividad['día_días'] = "No especificado"
                    dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
                    dias_encontrados = []
                    
                    # Primero buscar en tiny_text
                    if tiny_text:
                        contenido = tiny_text.get_text() 
                        dias_encontrados = [dia for dia in dias_semana if re.search(dia, contenido, re.IGNORECASE)]
                        
                    # Si no hay resultados, buscar también en info_actividad_fecha_lugar
                    if not dias_encontrados and info_actividad_fecha_lugar:
                        contenido_info = info_actividad_fecha_lugar.get_text()
                        dias_encontrados = [dia for dia in dias_semana if re.search(dia, contenido_info, re.IGNORECASE)]

                    # Asignar el resultado    
                    datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"

                    # Horario
                    datos_actividad['horario'] = "No especificado"
            
                    # def extraer_horario(texto_html):
                    #     soup = BeautifulSoup(str(texto_html), 'html.parser')
                    #     bloque = ""

                    #     # Buscar el tag que contenga la palabra "Horarios"
                    #     tag_horarios = None
                    #     for tag in soup.find_all(string=re.compile(r'\bHorarios\b', re.IGNORECASE)):
                    #         tag_horarios = tag.find_parent()
                    #         if tag_horarios:
                    #             siguiente_ul = None
                    #             for sibling in tag_horarios.find_all_next():
                    #                 if sibling.name == 'ul':
                    #                     siguiente_ul = sibling
                    #                     break
                    #             if siguiente_ul:
                    #                 items = [li.get_text(" ", strip=True) for li in siguiente_ul.find_all("li")]
                    #                 bloque = " ".join(items)
                    #                 bloque = re.sub(r'\s+', ' ', bloque).strip()
                    #                 if bloque:
                    #                     return bloque

                    #     # Si no encontramos un bloque, buscar patrones en texto plano como respaldo
                    #     texto = soup.get_text(separator=' ', strip=True)
                        
                    #     # Para "de XX:XX a YY:YY horas" o "de XX:XX a YY:YY h" o "de XX:XX a YY:YY h."
                    #     patron_rango = re.search(r'(de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                    #     if patron_rango:
                    #         return patron_rango.group(1).strip()
                        
                    #     # Para "de XX a YY horas" o "de XX a YY h" o "de XX a YY h."
                    #     patron_rango_simple = re.search(r'(de\s+\d{1,2}\s+a\s+\d{1,2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                    #     if patron_rango_simple:
                    #         return patron_rango_simple.group(1).strip()
                        
                    #     # Para "a las XX[:YY] horas" o "a las XX[:YY] h" o "a las XX[:YY] h."
                    #     patron_a_las = re.search(r'(a\s+las\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                    #     if patron_a_las:
                    #         return patron_a_las.group(1).strip()
                        
                    #     # Para "de XX a YY horas y de ZZ:ZZ a WW:WW horas" (o versiones con "h"/"h.")
                    #     patron_multiples = re.search(r'(de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b(?:\s+y\s+de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)?)', texto, re.IGNORECASE)
                    #     if patron_multiples:
                    #         return patron_multiples.group(1).strip()
                        
                    #     # Para fechas específicas seguidas de horario: "5, 11, 12... de abril de 10:30 a 13:30 h."
                    #     patron_fecha_hora = re.search(r'(\d+(?:,\s+\d+)*(?:\s+y\s+\d+)?\s+de\s+[a-zá-úñ]+\s+de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                    #     if patron_fecha_hora:
                    #         return patron_fecha_hora.group(1).strip()
                        
                    #     # Por si acaso, si contiene "horas"/"h"/"h." y algún formato de hora pero no encaja en los patrones anteriores
                    #     if re.search(r'\b(?:horas|h\.?)\b', texto.lower()) and re.search(r'\d{1,2}(?:[:.]\d{2})?', texto):
                    #         return texto.strip()
                        
                    #     return None

                    # # Buscar en tiny_text
                    # # if tiny_text:
                    # #     fragmentos = re.split(r'[.,;()\n]', tiny_text.get_text())
                    # #     for frag in fragmentos:
                    # #         horario_extraido = extraer_horario(frag)
                    # #         if horario_extraido:
                    # #             datos_actividad['horario'] = horario_extraido
                    # #             break
                    # if tiny_text:
                    #     horario_extraido = extraer_horario (tiny_text)
                    #     if horario_extraido:
                    #         datos_actividad['horario']= horario_extraido

                    # # Buscar en info_actividad_fecha_lugar si aún no se ha encontrado
                    # if datos_actividad['horario'] == "No especificado" and info_actividad_fecha_lugar:
                    #     fragmentos = re.split(r'[.,;()\n]', info_actividad_fecha_lugar.get_text())
                    #     for frag in fragmentos:
                    #         horario_extraido = extraer_horario(frag)
                    #         if horario_extraido:
                    #             datos_actividad['horario'] = horario_extraido
                    #             break

                    def extraer_horario(texto_html, buscar_bloque=False):
                        soup = BeautifulSoup(str(texto_html), 'html.parser')
                        bloque = ""

                        if buscar_bloque:
                            # Buscar el tag que contenga la palabra "Horarios:"
                            tag_horarios = None
                            for tag in soup.find_all(string=re.compile(r'\bHorarios:\b', re.IGNORECASE)):
                                tag_horarios = tag.find_parent()
                                if tag_horarios:
                                    siguiente_ul = None
                                    for sibling in tag_horarios.find_all_next():
                                        if sibling.name == 'ul':
                                            siguiente_ul = sibling
                                            break
                                    if siguiente_ul:
                                        items = [li.get_text(" ", strip=True) for li in siguiente_ul.find_all("li")]
                                        bloque = " ".join(items)
                                        bloque = re.sub(r'\s+', ' ', bloque).strip()
                                        if bloque:
                                            return bloque
                    

                        # Si no encontramos un bloque o no se busca, buscar patrones en texto plano como respaldo
                        texto = soup.get_text(separator=' ', strip=True)

                        # Para "de XX:XX a YY:YY horas" o "de XX:XX a YY:YY h" o "de XX:XX a YY:YY h."
                        patron_rango = re.search(r'(de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                        if patron_rango:
                            return patron_rango.group(1).strip()

                        # Para "de XX a YY horas" o "de XX a YY h" o "de XX a YY h."
                        patron_rango_simple = re.search(r'(de\s+\d{1,2}\s+a\s+\d{1,2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                        if patron_rango_simple:
                            return patron_rango_simple.group(1).strip()

                        # Para "a las XX[:YY] horas" o "a las XX[:YY] h" o "a las XX[:YY] h."
                        patron_a_las = re.search(r'(a\s+las\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                        if patron_a_las:
                            return patron_a_las.group(1).strip()

                        # Para "de XX a YY horas y de ZZ:ZZ a WW:WW horas" (o versiones con "h"/"h.")
                        patron_multiples = re.search(r'(de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b(?:\s+y\s+de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)?)', texto, re.IGNORECASE)
                        if patron_multiples:
                            return patron_multiples.group(1).strip()

                        # Para fechas específicas seguidas de horario: "5, 11, 12... de abril de 10:30 a 13:30 h."
                        patron_fecha_hora = re.search(r'(\d+(?:,\s+\d+)*(?:\s+y\s+\d+)?\s+de\s+[a-zá-úñ]+\s+de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
                        if patron_fecha_hora:
                            return patron_fecha_hora.group(1).strip()

                        # Por si acaso, si contiene "horas"/"h"/"h." y algún formato de hora pero no encaja en los patrones anteriores
                        if re.search(r'\b(?:horas|h\.?)\b', texto.lower()) and re.search(r'\d{1,2}(?:[:.]\d{2})?', texto):
                            return texto.strip()

                        return None

                    # Inicializar el campo horario
                    datos_actividad['horario'] = "No especificado"

                    # Buscar en info_actividad_fecha_lugar
                    if info_actividad_fecha_lugar:
                        fragmentos = re.split(r'[.,;()\n]', info_actividad_fecha_lugar.get_text())  #ojo aquí si aparecen .,()separando algo
                        for frag in fragmentos:
                            horario_extraido = extraer_horario(frag)
                            if horario_extraido:
                                datos_actividad['horario'] = horario_extraido
                                break

                    # Buscar en tiny_text si aún no se ha encontrado
                    if datos_actividad['horario'] == "No especificado" and tiny_text:
                        texto_tiny = tiny_text.get_text()
                        if re.search(r'\bHorarios:\b', texto_tiny, re.IGNORECASE):
                            horario_extraido = extraer_horario(tiny_text, buscar_bloque=True)
                            if horario_extraido:
                                datos_actividad['horario'] = horario_extraido
                        else:
                            fragmentos = re.split(r'[.,;()\n]', texto_tiny)
                            for frag in fragmentos:
                                horario_extraido = extraer_horario(frag)
                                if horario_extraido:
                                    datos_actividad['horario'] = horario_extraido
                                    break


                    # Fecha
                    datos_actividad['fecha'] = "Sin fecha"
                    if info_actividad_fecha_lugar:
                        # Buscar primero el encabezado h4 con clase "fecha title9"
                        fecha_header = info_actividad_fecha_lugar.find('h4', class_='fecha title9')
                        if fecha_header:
                            # Si lo encuentra, buscar el p.text-date inmediatamente después de este encabezado
                            # Primeramente, busca un p.text-date que estaría después del encabezado:
                            fecha_elem = fecha_header.find_next('p', class_='text-date')
                            if fecha_elem:
                                fecha_texto = fecha_elem.text.strip()
                                # Eliminar la parte de la hora si existe
                                if ' a las ' in fecha_texto:
                                    fecha_texto = fecha_texto.split(' a las ')[0].strip()
                                datos_actividad['fecha'] = fecha_texto
                            else:
                                # Método 2: Si no encuentra text-date, buscar div.tiny-text después del encabezado
                                tiny_text_div = fecha_header.find_next('div', class_='tiny-text')
                                if tiny_text_div:
                                    # Extraer el texto del primer párrafo en tiny-text que generalmente contiene la fecha
                                    fecha_parrafo = tiny_text_div.find('p')
                                    if fecha_parrafo:
                                        fecha_texto = fecha_parrafo.text.strip()
                                        # Eliminar la parte de la hora si existe
                                        if ' a las ' in fecha_texto:
                                            fecha_texto = fecha_texto.split(' a las ')[0].strip()
                                        datos_actividad['fecha'] = fecha_texto

                    # Lugar nombre
                    datos_actividad['lugar_nombre'] = "Sin lugar"
                    if info_actividad_fecha_lugar:
                        lugar_elem = info_actividad_fecha_lugar.find('a', class_='url fn')
                        if lugar_elem:
                            datos_actividad['lugar_nombre'] = lugar_elem.text.strip()

                    # Dirección
                    datos_actividad['lugar_dirección'] = "Sin dirección"
                    direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
                    if direccion_elem and direccion_elem.find('dd'):
                        direccion = direccion_elem.find('dd').text.strip()
                        datos_actividad['lugar_dirección'] = re.sub(r'\s+', ' ', direccion)

                    # Precio y recomendación
                    datos_actividad['precio'] = "No especificado"
                    datos_actividad['recomendación'] = "No especificado"
                    if actividades_info:
                        precio_elem = actividades_info.find('p', class_='gratuita')
                        if precio_elem:
                            datos_actividad['precio'] = precio_elem.text.strip()
                        recomendacion_elem = actividades_info.find('p', class_='ninos')
                        if recomendacion_elem:
                            datos_actividad['recomendación'] = recomendacion_elem.text.strip()
                    if datos_actividad['precio'] == "No especificado" and tiny_text:
                        precio_parrafos = []
                        # Opción 2: O tomar solo párrafos dentro de blockquotes
                        blockquotes = tiny_text.find_all('blockquote')
                        for bq in blockquotes:
                            for p in bq.find_all('p'):
                                if 'euro' in p.text.lower():
                                    precio_parrafos.append(p.text.strip())
                
                        if precio_parrafos:
                            datos_actividad['precio'] = ". ".join(precio_parrafos)
                            
                    

                    # URL ampliar información
                    datos_actividad['url_ampliar_info'] = "No disponible"
                    if tramites_content:
                        h4 = tramites_content.find('h4', class_='title8', string='Amplíe información')
                        if h4:
                            p = h4.find_next('p')
                            if p and p.find('a'):
                                url_info = p.find('a')['href']
                                datos_actividad['url_ampliar_info'] = "https://www.madrid.es" + url_info if not url_info.startswith('http') else url_info
                    
                    # Extraer URL de imagen embebida
                    if tramites_content:
                        imagen_elem = tramites_content.find('img')
                        if imagen_elem and imagen_elem.get('src'):
                            url_imagen = imagen_elem['src']
                            if not url_imagen.startswith('http'):
                                url_imagen = "https://www.madrid.es" + url_imagen
                            datos_actividad['url_imagen'] = url_imagen
                        else:
                            datos_actividad['url_imagen'] = "No disponible"
                    else:
                        datos_actividad['url_imagen'] = "No disponible"

                    
                    # Añadir URL de la página de detalle para referencia
                    datos_actividad['url_actividad'] = url_actividad
                    
                    # Imprimir los resultados en el formato exacto solicitado
                    print("\nDatos obtenidos de la actividad:")
                    for clave, valor in datos_actividad.items():
                        if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                            print(f"{clave}: {valor}")
                    
                    todas_actividades.append(datos_actividad)
                else:
                    print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")            
            else:
                print("No se encontró el enlace a la página de detalle.")
                        
        # Crear DataFrame con todas las actividades y guardar CSV
        if todas_actividades:
            df = pd.DataFrame(todas_actividades)
            directorio = '../data/raw/'
            nombre_archivo = f'{directorio}actividades_detalles_imagenes_pagina1_{fecha_hoy}.csv'
            df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"Datos guardados correctamente en '{nombre_archivo}'")
            print(f"Se han extraído datos de {len(todas_actividades)} actividades.")
        else:
            print("No se pudieron extraer datos de ninguna actividad.")
    else:
        print("No se encontró el contenedor principal de actividades.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")


Contenedor principal encontrado.
Consulta a fecha de 02-05-2025: se encontraron 25 actividades.

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: de 18:30 a 19:30 horas
fecha: Del miércoles 19 de

## Sección de pruebas con actividad en concreto
(probar alguna extracción problemática con actividad concreta en url)

In [113]:
url_actividad = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Campamentos-urbanos-de-verano-2025-Distrito-Arganzuela/?vgnextfmt=default&vgnextoid=0f5297c7fe556910VgnVCM2000001f4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD"
response_detalle = requests.get(url_actividad)

soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
contenedor_actividad = soup_detalle.find('div', class_='Panel 1.1')
tramites_content = contenedor_actividad.find('div', class_='tramites-content')
tiny_text = contenedor_actividad.find('div', class_='tiny-text')
actividades_info = contenedor_actividad.find('div', class_='actividades-info')
info_actividad_fecha_lugar = contenedor_actividad.find('div', class_='info-actividad')
def extraer_horario(texto_html, buscar_bloque=False):
    soup = BeautifulSoup(str(texto_html), 'html.parser')
    bloque = ""

    if buscar_bloque:
        # Buscar el tag que contenga la palabra "Horarios:"
        tag_horarios = None
        for tag in soup.find_all(string=re.compile(r'\bHorarios:\b', re.IGNORECASE)):
            tag_horarios = tag.find_parent()
            if tag_horarios:
                siguiente_ul = None
                for sibling in tag_horarios.find_all_next():
                    if sibling.name == 'ul':
                        siguiente_ul = sibling
                        break
                if siguiente_ul:
                    items = [li.get_text(" ", strip=True) for li in siguiente_ul.find_all("li")]
                    bloque = " ".join(items)
                    bloque = re.sub(r'\s+', ' ', bloque).strip()
                    if bloque:
                        return bloque


    # Si no encontramos un bloque o no se busca, buscar patrones en texto plano como respaldo
    texto = soup.get_text(separator=' ', strip=True)

    # Para "de XX:XX a YY:YY horas" o "de XX:XX a YY:YY h" o "de XX:XX a YY:YY h."
    patron_rango = re.search(r'(de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
    if patron_rango:
        return patron_rango.group(1).strip()

    # Para "de XX a YY horas" o "de XX a YY h" o "de XX a YY h."
    patron_rango_simple = re.search(r'(de\s+\d{1,2}\s+a\s+\d{1,2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
    if patron_rango_simple:
        return patron_rango_simple.group(1).strip()

    # Para "a las XX[:YY] horas" o "a las XX[:YY] h" o "a las XX[:YY] h."
    patron_a_las = re.search(r'(a\s+las\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
    if patron_a_las:
        return patron_a_las.group(1).strip()

    # Para "de XX a YY horas y de ZZ:ZZ a WW:WW horas" (o versiones con "h"/"h.")
    patron_multiples = re.search(r'(de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b(?:\s+y\s+de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)?)', texto, re.IGNORECASE)
    if patron_multiples:
        return patron_multiples.group(1).strip()

    # Para fechas específicas seguidas de horario: "5, 11, 12... de abril de 10:30 a 13:30 h."
    patron_fecha_hora = re.search(r'(\d+(?:,\s+\d+)*(?:\s+y\s+\d+)?\s+de\s+[a-zá-úñ]+\s+de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
    if patron_fecha_hora:
        return patron_fecha_hora.group(1).strip()

    # Por si acaso, si contiene "horas"/"h"/"h." y algún formato de hora pero no encaja en los patrones anteriores
    if re.search(r'\b(?:horas|h\.?)\b', texto.lower()) and re.search(r'\d{1,2}(?:[:.]\d{2})?', texto):
        return texto.strip()

    return None

# Inicializar el campo horario
datos_actividad['horario'] = "No especificado"

# Buscar en info_actividad_fecha_lugar
if info_actividad_fecha_lugar:
    fragmentos = re.split(r'[<>\n]', info_actividad_fecha_lugar.get_text())  #ojo aquí si aparecen .,()separando algo
    for frag in fragmentos:
        horario_extraido = extraer_horario(frag)
        if horario_extraido:
            datos_actividad['horario'] = horario_extraido
            break

# Buscar en tiny_text si aún no se ha encontrado
if datos_actividad['horario'] == "No especificado" and tiny_text:
    # texto_tiny = tiny_text.get_text()
    texto_tiny = tiny_text.get_text(separator=' ', strip=True)
    # texto_tiny = re.sub(r'\s+', ' ', texto_tiny)  # Reemplaza secuencias de espacios por un solo espacio
    texto_tiny = texto_tiny.replace('\xa0', ' ')  # Reemplaza específicamente los espacios no rompibles
    if re.search(r'\bHorarios:\b', texto_tiny, re.IGNORECASE):
        horario_extraido = extraer_horario(tiny_text, buscar_bloque=True)
        if horario_extraido:
            datos_actividad['horario'] = horario_extraido
    else:
        fragmentos = re.split(r'[.,;()\n]', texto_tiny)
        for frag in fragmentos:
            horario_extraido = extraer_horario(frag)
            if horario_extraido:
                datos_actividad['horario'] = horario_extraido
                break

In [112]:
re.search(r'Horarios:', texto_tiny, re.IGNORECASE) == True

False

In [114]:
tiny_text

<div class="tiny-text">
<p class="MsoNormal">Se realizarán actividades de ocio lúdico-educativas, incluyendo los servicios de <strong>desayuno y comida</strong>. La actividad será <strong>gratuita </strong>y se ofertarán 240 plazas durante el mes de julio y 150 plazas durante los días de junio y 180 plazas por centro durante el mes de agosto y septiembre. En total 1.170 plazas durante todo el verano. </p>
<p class="MsoNormal"><strong> Periodos:</strong></p>
<ul>
<li>TURNO 1: Del 23 al 27 de junio de 2025 (5 días)</li>
<li>TURNO 2: Del 30 de junio al 11 de julio de 2025 (10 días)</li>
<li>TURNO 3: Del 14 al 25 de julio de 2025 (9 días)</li>
<li>TURNO 4: Del 28 de julio al 8 de agosto de 2025 (10 días)</li>
<li>TURNO 5: Del 11 al 22 de agosto de 2025 (9 días)</li>
<li>TURNO 6: Del 25 de agosto al 5 de septiembre (10 días)<strong> </strong></li>
</ul>
<p class="MsoNormal"><strong> Centros educativos:</strong></p>
<ul>
<li class="MsoNormal"><strong>CEIP MIGUEL DE UNAMUNO,</strong> Calle de

In [115]:
texto_tiny

'Se realizarán actividades de ocio lúdico-educativas, incluyendo los servicios de desayuno y comida . La actividad será gratuita y se ofertarán 240 plazas durante el mes de julio y 150 plazas durante los días de junio y 180 plazas por centro durante el mes de agosto y septiembre. En total 1.170 plazas durante todo el verano. Periodos: TURNO 1: Del 23 al 27 de junio de 2025 (5 días) TURNO 2: Del 30 de junio al 11 de julio de 2025 (10 días) TURNO 3: Del 14 al 25 de julio de 2025 (9 días) TURNO 4: Del 28 de julio al 8 de agosto de 2025 (10 días) TURNO 5: Del 11 al 22 de agosto de 2025 (9 días) TURNO 6: Del 25 de agosto al 5 de septiembre (10 días) Centros educativos: CEIP MIGUEL DE UNAMUNO, Calle de Alicante, 5, 28045 Madrid CEIP PLÁCIDO DOMINGO , Calle del Tejo, 5, 28045 Madrid Horarios: De 7.30 a 9 horas: horario de acogida (incluye desayuno con entrada hasta las 8.45 horas) De 9 a 15 horas: horario de actividades (incluye comida) De 15 a 15.30 horas: primera salida De 15.30 a 16.30 hor

In [94]:
datos_actividad['horario']

'30 a 9 horas: horario de acogida'

In [ ]:
fragmentos

In [ ]:
df

## Recursividad: obtener datos de todas las actividades

Ahora procedemos para todas las paginas, todas la actividades publicadas. Para hacerlo optimo y mejorable encapsulo todo mi código en una función. 

In [29]:
# Encapsulado de mi código de extracción en una función
def extraer_info_por_actividad(url_actividad, indice=0, total=0):
    """
    Extrae información detallada de una actividad a partir de su URL.
    
    Args:
        url_actividad (str): URL de la página de detalle de la actividad
        indice (int, optional): Índice de la actividad. Por defecto 0.
        total (int, optional): Total de actividades. Por defecto 0.
        
    Returns:
        dict or None: Diccionario con los datos de la actividad, o None si hay error
    """
    if indice > 0 and total > 0:
        print(f"\nProcesando actividad {indice} de {total}...")
    
    print(f"Accediendo a la página de detalle: {url_actividad}")
    
    # Hacer una solicitud a la página de detalle con un pequeño retraso para no sobrecargar el servidor
    time.sleep(2)  # Esperar 2 segundos entre solicitudes
    response_detalle = requests.get(url_actividad)

    if response_detalle.status_code == 200:
        soup_detalle = BeautifulSoup(response_detalle.text, 'html.parser')
        
        # Diccionario para almacenar los datos de la actividad
        datos_actividad = {}

        contenedor_actividad = soup_detalle.find('div', class_='Panel 1.1')
        if not contenedor_actividad:
            print("No se encontró el contenedor de contenido de la actividad.")
            return None

        tramites_content = contenedor_actividad.find('div', class_='tramites-content')
        tiny_text = contenedor_actividad.find('div', class_='tiny-text')
        actividades_info = contenedor_actividad.find('div', class_='actividades-info')
        info_actividad_fecha_lugar = contenedor_actividad.find('div', class_='info-actividad')

        # Título
        titulo_elem = soup_detalle.find('h3', class_='summary-title')
        datos_actividad['título'] = titulo_elem.text.strip() if titulo_elem else "Sin título"

        # Descripción
        datos_actividad['descripción'] = tiny_text.p.text.strip() if tiny_text and tiny_text.p else "Sin descripción"

        # Edad
        datos_actividad['edad'] = "No especificada"

        # Función auxiliar para extraer el rango de edad de un texto
        def extraer_rango_edad(texto):
            # Patrón para "de X a Y años"
            patron_rango = re.search(r'de\s+(\d+)\s+a\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_rango:
                return f"de {patron_rango.group(1)} a {patron_rango.group(2)} años"
            
            # Patrón para "Edad: X y Y años" o similares con dos puntos
            if ':' in texto:
                partes = texto.split(':', 1)
                if 'años' in partes[1]:
                    # Extraer solo la parte que contiene la edad después de los dos puntos
                    edad_parte = partes[1].strip()
                    # Si hay texto adicional después de "años", cortarlo
                    if ' años' in edad_parte:
                        edad_parte = edad_parte[:edad_parte.find(' años') + 5]
                    return edad_parte
            
            # Patrón para "entre X y Y años"
            patron_entre = re.search(r'entre\s+(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_entre:
                return f"entre {patron_entre.group(1)} y {patron_entre.group(2)} años"

            # Patrón para "X y Y años" (normalmente consecutivos)
            patron_y = re.search(r'(\d+)\s+y\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_y:
                return f"{patron_y.group(1)} y {patron_y.group(2)} años"
                                        
            # Patrón para "a partir de X años"
            patron_a_partir = re.search(r'a\s+partir\s+de\s+(\d+)\s+años', texto, re.IGNORECASE)
            if patron_a_partir:
                return f"a partir de {patron_a_partir.group(1)} años"
            
            # Si contiene años y alguna palabra clave pero no encaja en patrones anteriores
            if 'años' in texto and any(palabra in texto.lower() for palabra in ['edad', 'niñ', 'de', 'a partir']):
                return texto.strip()
            
            return None

        # Buscar en blockquote dentro de tiny_text
        if tiny_text:
            for blockquote in tiny_text.find_all('blockquote'):
                if 'Edad:' in blockquote.text and 'años' in blockquote.text:
                    edad_extraida = extraer_rango_edad(blockquote.text)
                    if edad_extraida:
                        datos_actividad['edad'] = edad_extraida
                        break
            
            # Si aún no se ha encontrado, buscar en todo el texto de tiny_text
            if datos_actividad['edad'] == "No especificada":
                posibles_frases = re.split(r'[.,;:()\n]', tiny_text.get_text())
                for frase in posibles_frases:
                    edad_extraida = extraer_rango_edad(frase)
                    if edad_extraida:
                        datos_actividad['edad'] = edad_extraida
                        break

        # Buscar en tramites_content si aún no se ha encontrado
        if datos_actividad['edad'] == "No especificada" and tramites_content:
            posibles_frases = re.split(r'[.,;:()\n]', tramites_content.get_text())
            for frase in posibles_frases:
                edad_extraida = extraer_rango_edad(frase)
                if edad_extraida:
                    datos_actividad['edad'] = edad_extraida
                    break

        # Inscripción
        texto_inscripcion = ""
        inscripcion_parrafo = ""

        # Primero buscar en div.info-actividad si existe:
        if info_actividad_fecha_lugar:
            h4 = info_actividad_fecha_lugar.find('h4', class_='inscripcion')
            if h4:
                siguiente_p = h4.find_next('p', class_='text-date')
                if siguiente_p:
                    inscripcion_parrafo = siguiente_p.text.strip()

        # Después buscar también en tiny_text (independientemente de si ya encontramos algo)
        if tiny_text:
            match = re.search(r'([^.]*inscripción[^.]*\.)', tiny_text.get_text(), re.IGNORECASE)
            if match:
                texto_inscripcion = match.group(0).strip()
            
            # Además, buscar en párrafos específicos dentro de tiny_text
            for p in tiny_text.find_all('p'):
                # Verificar si contiene la palabra "inscr" en cualquier parte del párrafo
                if re.search(r'inscr', p.text, re.IGNORECASE):
                    # Si ya tenemos contenido de info_actividad, concatenamos
                    if texto_inscripcion:
                        texto_inscripcion += ". " + p.text.strip()
                    else:
                        texto_inscripcion = p.text.strip()
                    break

        # Asignar el valor final de inscripción según lo que se haya encontrado
        if inscripcion_parrafo or texto_inscripcion:
            # Si tenemos ambos contenidos, los concatenamos
            if inscripcion_parrafo and texto_inscripcion:
                datos_actividad['inscripción'] = inscripcion_parrafo + ". " + texto_inscripcion
            elif inscripcion_parrafo:
                datos_actividad['inscripción'] = inscripcion_parrafo
            else:
                datos_actividad['inscripción'] = texto_inscripcion
        else:
            datos_actividad['inscripción'] = "No especificada"

        # Periodicidad
        datos_actividad['periodicidad'] = "No especificada"
        if tiny_text:
            contenido = tiny_text.get_text()
            dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
            periodos = {'semana': 'semanal', 'mes': 'mensual', 'año': 'anual'}
            for dia in dias_semana:
                for periodo, valor in periodos.items():
                    if re.search(rf'un\s+{dia}\s+(al|cada)\s+{periodo}', contenido, re.IGNORECASE):
                        datos_actividad['periodicidad'] = valor
                        break
                if datos_actividad['periodicidad'] != "No especificada":
                    break
            if datos_actividad['periodicidad'] == "No especificada":
                patrones = ['diaria', 'semanal', 'quincenal', 'mensual']
                for patron in patrones:
                    if re.search(patron, contenido, re.IGNORECASE):
                        datos_actividad['periodicidad'] = patron
                        break

        # Día/días
        datos_actividad['día_días'] = "No especificado"
        dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
        dias_encontrados = []
        
        # Primero buscar en tiny_text
        if tiny_text:
            contenido = tiny_text.get_text() 
            dias_encontrados = [dia for dia in dias_semana if re.search(dia, contenido, re.IGNORECASE)]
            
        # Si no hay resultados, buscar también en info_actividad_fecha_lugar
        if not dias_encontrados and info_actividad_fecha_lugar:
            contenido_info = info_actividad_fecha_lugar.get_text()
            dias_encontrados = [dia for dia in dias_semana if re.search(dia, contenido_info, re.IGNORECASE)]

        # Asignar el resultado    
        datos_actividad['día_días'] = ", ".join(dias_encontrados) if dias_encontrados else "No especificado"

        # Extraer horario
        def extraer_horario(texto_html, buscar_bloque=False):
            soup = BeautifulSoup(str(texto_html), 'html.parser')
            bloque = ""

            if buscar_bloque:
                # Buscar el tag que contenga la palabra "Horarios:"
                tag_horarios = None
                for tag in soup.find_all(string=re.compile(r'\bHorarios:\b', re.IGNORECASE)):
                    tag_horarios = tag.find_parent()
                    if tag_horarios:
                        siguiente_ul = None
                        for sibling in tag_horarios.find_all_next():
                            if sibling.name == 'ul':
                                siguiente_ul = sibling
                                break
                        if siguiente_ul:
                            items = [li.get_text(" ", strip=True) for li in siguiente_ul.find_all("li")]
                            bloque = " ".join(items)
                            bloque = re.sub(r'\s+', ' ', bloque).strip()
                            if bloque:
                                return bloque

            # Si no encontramos un bloque o no se busca, buscar patrones en texto plano como respaldo
            texto = soup.get_text(separator=' ', strip=True)

            # Para "de XX:XX a YY:YY horas" o "de XX:XX a YY:YY h" o "de XX:XX a YY:YY h."
            patron_rango = re.search(r'(de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_rango:
                return patron_rango.group(1).strip()

            # Para "de XX a YY horas" o "de XX a YY h" o "de XX a YY h."
            patron_rango_simple = re.search(r'(de\s+\d{1,2}\s+a\s+\d{1,2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_rango_simple:
                return patron_rango_simple.group(1).strip()

            # Para "a las XX[:YY] horas" o "a las XX[:YY] h" o "a las XX[:YY] h."
            patron_a_las = re.search(r'(a\s+las\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_a_las:
                return patron_a_las.group(1).strip()

            # Para "de XX a YY horas y de ZZ:ZZ a WW:WW horas" (o versiones con "h"/"h.")
            patron_multiples = re.search(r'(de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b(?:\s+y\s+de\s+\d{1,2}(?:[:.]\d{2})?\s+a\s+\d{1,2}(?:[:.]\d{2})?\s+(?:horas|h\.?)\b)?)', texto, re.IGNORECASE)
            if patron_multiples:
                return patron_multiples.group(1).strip()

            # Para fechas específicas seguidas de horario: "5, 11, 12... de abril de 10:30 a 13:30 h."
            patron_fecha_hora = re.search(r'(\d+(?:,\s+\d+)*(?:\s+y\s+\d+)?\s+de\s+[a-zá-úñ]+\s+de\s+\d{1,2}[:.]\d{2}\s+a\s+\d{1,2}[:.]\d{2}\s+(?:horas|h\.?)\b)', texto, re.IGNORECASE)
            if patron_fecha_hora:
                return patron_fecha_hora.group(1).strip()

            # Por si acaso, si contiene "horas"/"h"/"h." y algún formato de hora pero no encaja en los patrones anteriores
            if re.search(r'\b(?:horas|h\.?)\b', texto.lower()) and re.search(r'\d{1,2}(?:[:.]\d{2})?', texto):
                return texto.strip()

            return None

        # Inicializar el campo horario
        datos_actividad['horario'] = "No especificado"

        # Buscar en info_actividad_fecha_lugar
        if info_actividad_fecha_lugar:
            fragmentos = re.split(r'[.,;()\n]', info_actividad_fecha_lugar.get_text())
            for frag in fragmentos:
                horario_extraido = extraer_horario(frag)
                if horario_extraido:
                    datos_actividad['horario'] = horario_extraido
                    break

        # Buscar en tiny_text si aún no se ha encontrado
        if datos_actividad['horario'] == "No especificado" and tiny_text:
            texto_tiny = tiny_text.get_text()
            if re.search(r'\bHorarios:\b', texto_tiny, re.IGNORECASE):
                horario_extraido = extraer_horario(tiny_text, buscar_bloque=True)
                if horario_extraido:
                    datos_actividad['horario'] = horario_extraido
            else:
                fragmentos = re.split(r'[.,;()\n]', texto_tiny)
                for frag in fragmentos:
                    horario_extraido = extraer_horario(frag)
                    if horario_extraido:
                        datos_actividad['horario'] = horario_extraido
                        break

        # Fecha
        datos_actividad['fecha'] = "Sin fecha"
        if info_actividad_fecha_lugar:
            # Buscar primero el encabezado h4 con clase "fecha title9"
            fecha_header = info_actividad_fecha_lugar.find('h4', class_='fecha title9')
            if fecha_header:
                # Si lo encuentra, buscar el p.text-date inmediatamente después de este encabezado
                fecha_elem = fecha_header.find_next('p', class_='text-date')
                if fecha_elem:
                    fecha_texto = fecha_elem.text.strip()
                    # Eliminar la parte de la hora si existe
                    if ' a las ' in fecha_texto:
                        fecha_texto = fecha_texto.split(' a las ')[0].strip()
                    datos_actividad['fecha'] = fecha_texto
                else:
                    # Método 2: Si no encuentra text-date, buscar div.tiny-text después del encabezado
                    tiny_text_div = fecha_header.find_next('div', class_='tiny-text')
                    if tiny_text_div:
                        # Extraer el texto del primer párrafo en tiny-text que generalmente contiene la fecha
                        fecha_parrafo = tiny_text_div.find('p')
                        if fecha_parrafo:
                            fecha_texto = fecha_parrafo.text.strip()
                            # Eliminar la parte de la hora si existe
                            if ' a las ' in fecha_texto:
                                fecha_texto = fecha_texto.split(' a las ')[0].strip()
                            datos_actividad['fecha'] = fecha_texto

        # Lugar nombre
        datos_actividad['lugar_nombre'] = "Sin lugar"
        if info_actividad_fecha_lugar:
            lugar_elem = info_actividad_fecha_lugar.find('a', class_='url fn')
            if lugar_elem:
                datos_actividad['lugar_nombre'] = lugar_elem.text.strip()

        # Dirección
        datos_actividad['lugar_dirección'] = "Sin dirección"
        direccion_elem = soup_detalle.find('dl', class_='dl-horz adr')
        if direccion_elem and direccion_elem.find('dd'):
            direccion = direccion_elem.find('dd').text.strip()
            datos_actividad['lugar_dirección'] = re.sub(r'\s+', ' ', direccion)

        # Precio y recomendación
        datos_actividad['precio'] = "No especificado"
        datos_actividad['recomendación'] = "No especificado"
        if actividades_info:
            precio_elem = actividades_info.find('p', class_='gratuita')
            if precio_elem:
                datos_actividad['precio'] = precio_elem.text.strip()
            recomendacion_elem = actividades_info.find('p', class_='ninos')
            if recomendacion_elem:
                datos_actividad['recomendación'] = recomendacion_elem.text.strip()
        if datos_actividad['precio'] == "No especificado" and tiny_text:
            precio_parrafos = []
            # Opción 2: O tomar solo párrafos dentro de blockquotes
            blockquotes = tiny_text.find_all('blockquote')
            for bq in blockquotes:
                for p in bq.find_all('p'):
                    if 'euro' in p.text.lower():
                        precio_parrafos.append(p.text.strip())
    
            if precio_parrafos:
                datos_actividad['precio'] = ". ".join(precio_parrafos)

        # URL ampliar información
        datos_actividad['url_ampliar_info'] = "No disponible"
        if tramites_content:
            h4 = tramites_content.find('h4', class_='title8', string='Amplíe información')
            if h4:
                p = h4.find_next('p')
                if p and p.find('a'):
                    url_info = p.find('a')['href']
                    datos_actividad['url_ampliar_info'] = "https://www.madrid.es" + url_info if not url_info.startswith('http') else url_info
        
        # Extraer URL de imagen embebida
        if tramites_content:
            imagen_elem = tramites_content.find('img')
            if imagen_elem and imagen_elem.get('src'):
                url_imagen = imagen_elem['src']
                if not url_imagen.startswith('http'):
                    url_imagen = "https://www.madrid.es" + url_imagen
                datos_actividad['url_imagen'] = url_imagen
            else:
                datos_actividad['url_imagen'] = "No disponible"
        else:
            datos_actividad['url_imagen'] = "No disponible"

        
        # Añadir URL de la página de detalle para referencia
        datos_actividad['url_actividad'] = url_actividad
        
        # Imprimir los resultados en el formato exacto solicitado
        print("\nDatos obtenidos de la actividad:")
        for clave, valor in datos_actividad.items():
            if clave != 'url_actividad':  # No mostrar la url_actividad para mantener el formato original
                print(f"{clave}: {valor}")
        
        return datos_actividad
    else:
        print(f"Error al acceder a la página de detalle: código {response_detalle.status_code}")
        return None

probemos dicha función en la página 1

In [30]:
# llamado a la función desde una pagina, para ver si todo funciona igual y bien.

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time

# URL de la primera pagina, en la que trabajaré primeramente
url = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=1"

# Obtener la fecha de hoy
fecha_hoy = date.today().strftime("%d-%m-%Y")

response = requests.get(url)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    contenedor_principal = soup.find('ul', class_='events-results docs')
    
    if contenedor_principal:
        print("Contenedor principal encontrado.")
        
        # Buscar todas las actividades dentro del contenedor
        actividades = contenedor_principal.find_all('div', class_='event-info')
        
        print(f"Consulta a fecha de {fecha_hoy}: se encontraron {len(actividades)} actividades.")
       
        # Iterar sobre todas las actividades encontradas
        for indice, actividad in enumerate(actividades, 1):
            # Extraer el enlace de la actividad
            elemento_enlace = actividad.find('a', class_='event-link')
            if elemento_enlace and 'href' in elemento_enlace.attrs:
                url_actividad = elemento_enlace['href']
                if not url_actividad.startswith('http'):
                    url_actividad = "https://www.madrid.es" + url_actividad
                
                # Llamar a la función para extraer la información de la actividad
                datos_actividad = extraer_info_por_actividad(url_actividad, indice, len(actividades))
                
                # Si se encontraron datos, añadirlos a la lista
                if datos_actividad:
                    todas_actividades.append(datos_actividad)
            else:
                print("No se encontró el enlace a la página de detalle.")

Contenedor principal encontrado.
Consulta a fecha de 02-05-2025: se encontraron 25 actividades.

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Taller-de-ajedrez-avanzado-infantil-juvenil-Biblioteca-Maria-Lejarraga/?vgnextfmt=default&vgnextoid=991163008c7c5910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Taller de ajedrez avanzado (infantil-juvenil) Biblioteca María Lejárraga
descripción: Taller avanzado para el perfeccionamiento de la práctica del ajedrez, deporte de reconocidos beneficios a nivel cognitivo e intelectual que mejora la memoria, la concentración y permite ejercitar ambos hemisferios cerebrales.
edad: de 6 a 17 años
inscripción: Para participar es impresindible realizar inscripción previa en este enlace.
periodicidad: semanal
día_días: miércoles
horario: de 18:30 a 19:30 horas
fecha: Del miércoles 19 de

Ahora hago un código que recorra todas las páginas y obtenga los enlaces de todas las actividades. 

Ahora voy a completar el proceso para todas las páginas y mejorar algunas cosas

In [31]:
#importación de librerías
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import date
import re
import time


In [ ]:
# La función extraer_info_por_actividad se mantiene (ya está desarrollada y funciona correctamente)
# en celda superior

In [32]:
# Función para procesar las actividades de una pagina (de cada pagina, la se vaya iterando)
def procesar_pagina_actividades(url_pagina, numero_pagina, total_paginas):
    """
    Procesa una página de actividades y devuelve los datos de cada actividad.
    
    Args:
        url_pagina (str): URL de la página a procesar
        numero_pagina (int): Número de página actual
        total_paginas (int): Número total de páginas
    
    Returns:
        list: Lista de diccionarios con los datos de las actividades
    """
    print(f"\nAccediendo a la página {numero_pagina+1}/{total_paginas}: {url_pagina}")
    actividades_pagina = []
    
    try:
        # Pequeña pausa para no sobrecargar el servidor
        time.sleep(1)
        response_pagina = requests.get(url_pagina)
        
        if response_pagina.status_code == 200:
            soup_pagina = BeautifulSoup(response_pagina.text, 'html.parser')
            contenedor_actividades = soup_pagina.find('ul', class_='events-results docs')
            
            if contenedor_actividades:
                # Buscar todas las actividades dentro del contenedor
                actividades = contenedor_actividades.find_all('div', class_='event-info')
                print(f"Encontradas {len(actividades)} actividades en la página {numero_pagina+1}")
                
                # Iterar sobre todas las actividades encontradas
                for indice, actividad in enumerate(actividades, 1):
                    # Extraer el enlace de la actividad
                    elemento_enlace = actividad.find('a', class_='event-link')
                    if elemento_enlace and 'href' in elemento_enlace.attrs:
                        url_actividad = elemento_enlace['href']
                        if not url_actividad.startswith('http'):
                            url_actividad = "https://www.madrid.es" + url_actividad
                        
                        # Llamar a la función para extraer la información de la actividad
                        datos_actividad = extraer_info_por_actividad(url_actividad, indice, len(actividades))
                        
                        # Si se encontraron datos, añadirlos a la lista
                        if datos_actividad:
                            actividades_pagina.append(datos_actividad)
                    else:
                        print("No se encontró el enlace a la página de detalle.")
            else:
                print(f"No se encontró el contenedor de actividades en la página {numero_pagina+1}")
        else:
            print(f"Error al acceder a la página {numero_pagina+1}: código {response_pagina.status_code}")
    except Exception as e:
        print(f"Error inesperado al procesar la página {numero_pagina+1}: {str(e)}")
    
    return actividades_pagina

In [33]:
# Función obtener cuales paginas, una o todas, y cuales actividades, rango o todas, que el usuario quiere extraer
def procesar_rango_actividades(todas_actividades, inicio, fin):
    """
    Devuelve un subconjunto de actividades basado en índices de inicio y fin.
    
    Args:
        todas_actividades (list): Lista completa de actividades
        inicio (int): Índice de la primera actividad a incluir
        fin (int): Índice de la última actividad a incluir (inclusive)
    
    Returns:
        list: Subconjunto de actividades
    """
    # Validar índices
    if inicio < 1:
        inicio = 1
    if fin > len(todas_actividades):
        fin = len(todas_actividades)
    
    print(f"Filtrando actividades: se guardarán solo desde índice {inicio} hasta {fin} (de {len(todas_actividades)} totales)")
    
    
    print(f"Procesando actividades en el rango {inicio} a {fin} de {len(todas_actividades)}")
    return todas_actividades[inicio-1:fin]


In [34]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_imagen,url_actividad
0,Taller de ajedrez avanzado (infantil-juvenil) ...,Taller avanzado para el perfeccionamiento de l...,de 6 a 17 años,Para participar es impresindible realizar insc...,semanal,miércoles,de 18:30 a 19:30 horas,Del miércoles 19 de marzo de 2025 al miércoles...,Biblioteca Pública Municipal María Lejárraga (...,"CALLE PRINCESA DE EBOLI, 29 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
1,Manualidades,"Taller de manualidades ideado, coordinado y ej...",de 6 a 12 años,El proceso de inscripción finaliza el 23/06/20...,mensual,martes,de 18 a 19:30 horas,Del martes 25 de marzo de 2025 al martes 24 de...,Biblioteca Pública Municipal Gabriel García Má...,"PLAZA PUEBLO, 2 (CENTRO CULTURAL ORCASUR) 2804...",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
2,Talleres de Price (De 7 a 9 años),¡Diviértete aprendiendo mientras descubres tod...,de 7 a 9 años,No especificada,No especificada,sábado,De 10 a 12 horas,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadWeb/UGBBDD/MadridD...,https://www.madrid.es/portales/munimadrid/es/I...
3,Talleres del Price (5 y 6 años),¡Diviértete aprendiendo mientras descubres tod...,5 y 6 años,No especificada,No especificada,sábado,De 10 a 12 h,Del sábado 29 de marzo de 2025 al sábado 14 de...,Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",Abono trimestral (10 sesiones): 140 euros. Abo...,No especificado,No disponible,https://www.madrid.es/UnidadWeb/UGBBDD/MadridD...,https://www.madrid.es/portales/munimadrid/es/I...
4,HOP!,Los Absurdos Teatro nos traen un paseo teatral...,sus anécdotas y su importancia como centro cul...,No especificada,No especificada,No especificado,a las 11 horas,"20 de abril, 18 de mayo y 15 de junio de 2025.",Teatro Circo Price,"RONDA ATOCHA, 35 28012 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadWeb/UGBBDD/MadridD...,https://www.madrid.es/portales/munimadrid/es/I...
5,Campamentos urbanos de verano 2025. Distrito A...,Se realizarán actividades de ocio lúdico-educa...,Niños y niñas de 4 años a 12 años,No especificada,No especificada,domingo,30 a 9 horas: horario de acogida,De junio a agosto de 2025,Colegio Público Plácido Domingo,"CALLE TEJO, 5 28045 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadWeb/UGBBDD/Activid...,https://www.madrid.es/portales/munimadrid/es/I...
6,Ronda de libros,Este trimestre los participantes harán una exc...,de 1 a 3 años,Inscripciones.,No especificada,sábado,11:30 horas,"Sábados 26 de abril, 10 de mayo y 7 de junio 2...",Casa del Lector,"PASEO CHOPERA, 14 28045 MADRID",No especificado,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadWeb/UGBBDD/MadridD...,https://www.madrid.es/portales/munimadrid/es/I...
7,Talleres para niños de 4 a 14 años,,de 4 a 14 años,El proceso de inscripción ha finalizado,No especificada,sábado,No especificado,Del sábado 26 de abril de 2025 al sábado 28 de...,Centro Sociocultural Juvenil de Moratalaz,"CALLE FUENTE CARRANTONA, 10 28030 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
8,Campamento urbano de Semana Santa. Distrito Re...,Campamento urbano de verano 2025 ofertado por ...,No especificada,El proceso de inscripción comienza el 06/05/20...,No especificada,"lunes, viernes",De 9 a 16:30 horas,Del martes 29 de abril de 2025 al viernes 5 de...,Colegio Público

In [35]:
# CELDA PRINCIPAL PARA EJECUTAR Y LLAMAR A LAS FUNCIONES ANIDADAS
# URL base
url_base = "https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD"

# Obtener fecha actual
fecha_hoy = date.today().strftime("%d-%m-%Y")

# Primera solicitud para obtener el total
response = requests.get(url_base)
if response.status_code == 200:
    print("Correcto: el servidor respondió con código 200.")
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extraer el número total de actividades
    contenedor_principal = soup.find('ul', id='totalResultsUL')
    total_actividades = 0
    if contenedor_principal:
        total_elementos = contenedor_principal.find('strong')
        if total_elementos:
            total_actividades = int(total_elementos.text)
            print(f"Consulta fecha {fecha_hoy}. Número total de actividades: {total_actividades}")
        else:
            print("No se encontró el elemento que contiene el número total")
    else:
        print("No se encontró el contenedor de resultados totales")
    
    # Calcular número total de páginas
    if total_actividades % 25 == 0:
        total_paginas = total_actividades // 25
    else:
        total_paginas = (total_actividades // 25) + 1
    print(f"Número total de páginas a recorrer: {total_paginas}")
    print("Nota: La página 1 corresponde al valor page=0 de la web")
    
    # Preguntar qué páginas procesar
    while True:
        pagina_input = input(f"\nIntroduce la página a consultar (1-{total_paginas}) o 't' para todas: ")
        
        if pagina_input.lower() == 't':
            pagina_inicio = 0
            pagina_fin = total_paginas
            print(f"Se procesarán todas las páginas (1 a {total_paginas})")
            break
        else:
            try:
                pagina_num = int(pagina_input)-1
                if 0 <= pagina_num < total_paginas:
                    pagina_inicio = pagina_num
                    pagina_fin = pagina_num + 1
                    print(f"Se procesará solo la página {pagina_num +1}")
                    break
                else:
                    print(f"Error: El número debe estar entre 1 y {total_paginas}")
            except ValueError:
                print("Error: Introduce un número válido o 't'")
    
    # Preguntar si se quiere procesar un rango específico de actividades
    procesar_rango = input("\nDentro de esta página, ¿deseas procesar solo un rango específico de actividades? (entre 1 y 25) (s/n): ")
    inicio_actividad = 0
    fin_actividad = None  # Valor None indica procesar todas
    
    if procesar_rango.lower() == 's':
        try:
            inicio_actividad = int(input("Introduce el índice de la primera actividad a procesar: "))
            fin_actividad = int(input("Introduce el índice de la última actividad a procesar (Enter para todas): "))

            if fin_actividad <= 25:  
                print(f"Se procesarán actividades desde índice {inicio_actividad} hasta {fin_actividad}")
            else:
                print(f"Se procesarán todas las actividades desde el índice {inicio_actividad}")
            
        except ValueError:
            print("Valores no válidos. Se procesarán todas las actividades.")
            inicio_actividad = 1
            fin_actividad = None
    
    # Lista para almacenar todas las actividades
    todas_actividades = []
    
    # Recorrer las páginas seleccionadas
    for pagina in range(pagina_inicio, pagina_fin):
        url_pagina = f"{url_base}&page={pagina}"
        actividades_pagina = procesar_pagina_actividades(url_pagina, pagina, total_paginas)
        todas_actividades.extend(actividades_pagina)
    
    # Aplicar el filtro de rango si es necesario
    actividades_a_guardar = todas_actividades
    if fin_actividad is not None:
        actividades_a_guardar = procesar_rango_actividades(todas_actividades, inicio_actividad, fin_actividad)
    
    # Crear DataFrame con las actividades seleccionadas y guardar CSV
    if actividades_a_guardar:
        df = pd.DataFrame(actividades_a_guardar)
        directorio = '../data/raw/'
        # Comprobar si el directorio existe, si no, crearlo
        os.makedirs(directorio, exist_ok=True)
        
        # Preparar nombre del archivo según el formato solicitado
        if pagina_inicio == 0 and pagina_fin == total_paginas:
            paginas_str = "todas"
        else:
            paginas_str = str(pagina_fin - pagina_inicio)
            
        if fin_actividad is not None:
            act_str = f"act{inicio_actividad+1}-{fin_actividad+1}"
        else:
            act_str = f"act{len(actividades_a_guardar)}"
            
        nombre_archivo = f'{directorio}actividades_detalle_página{paginas_str}_{act_str}_{fecha_hoy}.csv'
        
        df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
        print(f"\nDatos guardados correctamente en '{nombre_archivo}'")
        print(f"Se han extraído datos de {len(actividades_a_guardar)} actividades.")
    else:
        print("\nNo se pudieron extraer datos de ninguna actividad.")
else:
    print(f"Error: el servidor respondió con código {response.status_code}.")

Correcto: el servidor respondió con código 200.
Consulta fecha 02-05-2025. Número total de actividades: 261
Número total de páginas a recorrer: 11
Nota: La página 1 corresponde al valor page=0 de la web
Se procesará solo la página 5

Accediendo a la página 5/11: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Actividades-infantiles/?vgnextfmt=default&vgnextoid=fdc579db15034710VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD&page=4
Encontradas 25 actividades en la página 5

Procesando actividad 1 de 25...
Accediendo a la página de detalle: https://www.madrid.es/portales/munimadrid/es/Inicio/Cultura-ocio-y-deporte/Orcobblu/?vgnextfmt=default&vgnextoid=d6ac03a1c2d56910VgnVCM1000001d4a900aRCRD&vgnextchannel=7911f073808fe410VgnVCM2000000c205a0aRCRD

Datos obtenidos de la actividad:
título: Orcobblu
descripción: Edad recomendada: +5 años
edad: No especificada
inscripción: No especificada
periodicidad: No especificada
día_días: domingo

In [37]:
df.sample(8)

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_imagen,url_actividad
15,Five little goats,Vamos a leer en inglés y a divertirnos en la b...,a partir de 4 años,No especificada,No especificada,lunes,a las 18 horas,Lunes 12 de mayo de 2025,Biblioteca Pública Municipal Mario Vargas Llos...,"CALLE BARCELO, 4 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
21,Visita externa,Para adultos. Máximo 20 plazas.,No especificada,No especificada,No especificada,martes,a las 11 horas,Martes 13 de mayo de 2025,Centro de Interpretación de la Naturaleza Mont...,"Carretera M-607 Km 13 L-10, null MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
17,'Animalitis',"Duendes, tesoros, aventuras, misterios... Las ...",a partir de 4 años,No especificada,No especificada,martes,a las 17:30 horas,Martes 13 de mayo de 2025,Biblioteca Pública Municipal María Zambrano (T...,"PLAZA DONOSO, 5 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
10,Juego de pistas 'Los animales del bosque',Dinámicas participativas para grupos familiare...,de 4 a 7 años,Más información en el apartado inscripción.. S...,No especificada,domingo,a las 11 horas,Domingo 11 de mayo de 2025,Aula ambiental La Cabaña del Retiro,"PASEO FERNAN NUÑEZ, 10 28009 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
3,Cuentos con amor por un mundo mejor,"Duendes, tesoros, aventuras, misterios... Las ...",a partir de 3 años,No especificada,No especificada,lunes,a las 12 horas,Domingo 11 de mayo de 2025,Biblioteca Pública Municipal Ángel González (L...,"CALLE GRANJA DE TORREHERMOSA, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
23,'Cuentos en la granja',"Duendes, tesoros, aventuras, misterios... Las ...",a partir de 4 años,Para asistir es necesario realizar inscripción...,No especificada,miércoles,a las 18 horas,Miércoles 14 de mayo de 2025,Biblioteca Pública Municipal Aluche (Latina),"CALLE CAMARENA, 10 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...
7,Talleres Infantiles,Para menores de 6 a 8 años. Máximo 20 plazas.,de 6 a 8 años,No especificada,No especificada,domingo,De 11 a 12 h,Del domingo 11 de mayo de 2025 al domingo 18 d...,Centro de Interpretación de la Naturaleza Mont...,"Carretera M-607 Km 13 L-10, null MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
4,El dragón Nabú,,Recomendado para mayores de 4 años,No especificada,No especificada,domingo,a las 12 horas,Domingo 11 de mayo de 2025,Auditorio Joaquín Rosado (Moncloa-Aravaca),"CALLE SANTA POLA, 22 28008 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/UnidadesDescentralizadas...,https://www.madrid.es/portales/munimadrid/es/I...


## Trabajando en el csv completo obtenido

Ahora voy a desglosar algunos datos y organizarlos hasta lograr un Dataframe más manejable y fácil de visualizar. Para ello voy a abrir el archivo obtenido antes

In [39]:
# seleccionar la fecha del csv
import os
import pandas as pd
# Definir ruta del archivo
directorio = '../data/raw/'
nombre_archivo_trabajo = 'actividades_detalle_páginatodas_act250_24-04-2025.csv'
ruta_archivo_trabajo = os.path.join(directorio, nombre_archivo_trabajo)

df = pd.read_csv(ruta_archivo_trabajo)

In [40]:
df

,título,descripción,edad,inscripción,periodicidad,día_días,horario,fecha,lugar_nombre,lugar_dirección,precio,recomendación,url_ampliar_info,url_actividad
0,Talleres creativos en idioma serbio,Talleres creativos para niños y niñas de 3 a16...,Talleres creativos para niños y niñas de 3 a16...,No especificada,No especificada,domingo,de 11 a 13 horas,Del domingo 22 de septiembre de 2024 al doming...,Biblioteca Pública Municipal Iván de Vargas (C...,"CALLE SAN JUSTO, 5 28005 MADRID",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
1,Tertulias en Inglés (infantil) Biblioteca Vall...,Sin descripción,de 8 a 12 años,Para participar es necesario realizar inscripc...,semanal,miércoles,de 18 a 19:30 horas,Del miércoles 25 de septiembre de 2024 al miér...,Biblioteca Pública Municipal Vallecas (Puente ...,"CALLE PUERTO DEL MONASTERIO, 1 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
2,Club de lectura (infantil) Biblioteca David Gi...,"Una de nuestras actividades favoritas, dónde l...",de 9 a 12 años,Para participar es impresindible realizar insc...,quincenal,lunes,de 18 a 19:30 horas,Del lunes 30 de septiembre de 2024 al lunes 9 ...,Biblioteca Pública Municipal David Gistau (Sal...,"AVENIDA TOREROS, 5 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
3,Taller de cómic (Infantil) Biblioteca Pozo del...,Son talleres de creación dirigidos a dar a con...,de 8 a 11 años,Para participar es necesario realizar inscripc...,quincenal,lunes,de 17:30 a 19 horas,Del lunes 30 de septiembre de 2024 al lunes 2 ...,Biblioteca Pública Municipal Pozo del Tío Raim...,"AVENIDA GLORIETAS, 19 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
4,Club de lectura (infantil) Biblioteca San Fermín,"Una de nuestras actividades favoritas, dónde l...",de 9 a 12 años,Para participar es impresindible realizar insc...,quincenal,miércoles,de 18 a 19:30 horas,Del miércoles 2 de octubre de 2024 al miércole...,Biblioteca Pública Municipal San Fermín (Usera),"AVENIDA SAN FERMIN, 10 MADRID",Gratuito,Recomendado para niñas y niños,https://www.madrid.es/portales/munimadrid/es/I...,https://www.madrid.es/portales/munimadrid/es/I...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Campamento de Verano 2025 - Distrito de Villa...,Se realizarán actividades de ocio lúdico-educa...,No especificada,No especificada,No especificada,"lunes, jueves, viernes",No especificado,Del lunes 23 de junio de 2025 al jueves 31 de ...,Sin lugar,Sin dirección,Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
246,Campamentos de verano 2025 en Puente de Vallecas,"Se ofertan 250 plazas por cada quincena, para ...",para menores nacidos entre los años 2013 y 202...,"La inscripción es gratuita, realizándose un so...",No especificada,No especificado,a las 17 horas,El campamento se celebrará en los CEIP Fray Ju...,Sin lugar,Sin dirección,Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
247,Verde que te quiero verde - Zaguán Teatro (Cas...,"En 1923, un joven Federico García Lorca decidi...",Cien años han pasado de la presentación,No especificada,No especificada,"sábado, domingo",Horario: 18:30 horas,Del sábado 28 de junio de 2025 al domingo 29 d...,Teatro Municipal de Títeres. Parque de El Retiro,"AVENIDA MÉXICO, 4 (Parque de El Retiro. Entrad...",Gratuito,Recomendado para niñas y niños,No disponible,https://www.madrid.es/portales/munimadrid/es/I...
248,Los más pequeños también plantamos,NaN,No especificada,No especificada,No especificada,domingo,a las 11 horas,Domingo 29 de junio de 2025,Centro de Información y Educación Ambiental de...,"PASEO FERN

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   título            250 non-null    object
 1   descripción       240 non-null    object
 2   edad              250 non-null    object
 3   inscripción       250 non-null    object
 4   periodicidad      250 non-null    object
 5   día_días          250 non-null    object
 6   horario           250 non-null    object
 7   fecha             250 non-null    object
 8   lugar_nombre      250 non-null    object
 9   lugar_dirección   250 non-null    object
 10  precio            250 non-null    object
 11  recomendación     250 non-null    object
 12  url_ampliar_info  250 non-null    object
 13  url_actividad     250 non-null    object
dtypes: object(14)
memory usage: 27.5+ KB


In [42]:
df['recomendación'].unique()

array(['Recomendado para niñas y niños', 'No especificado'], dtype=object)

In [ ]:
# # Importamos las librerías necesarias
# import pandas as pd
# import numpy as np
# from datetime import datetime
# import re
# from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
# from sklearn.feature_extraction.text import TfidfVectorizer


# # 1. Limpieza de datos
# # ------------------

# # Manejar valores faltantes (reemplazar "No especificada" y "No disponible" por NaN)
# df.replace(["No especificada", "No especificado", "No disponible"], np.nan, inplace=True)

# # 2. Procesamiento de columnas de texto
# # ----------------------------------

# # Procesamiento de título: extraer longitud y crear vectores TF-IDF
# df['titulo_longitud'] = df['título'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

# # Procesamiento de descripción: extraer longitud y crear vectores TF-IDF
# df['descripcion_longitud'] = df['descripción'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

# # TF-IDF para título y descripción (reducido a 5 componentes para cada uno)
# tfidf_titulo = TfidfVectorizer(max_features=5, stop_words=['de', 'la', 'el', 'en', 'y', 'a', 'para'])
# tfidf_desc = TfidfVectorizer(max_features=5, stop_words=['de', 'la', 'el', 'en', 'y', 'a', 'para'])

# # Solo aplicamos TF-IDF si tenemos suficientes datos no nulos
# if df['título'].notna().sum() > 1:
#     titulo_tfidf = tfidf_titulo.fit_transform(df['título'].fillna(''))
#     titulo_tfidf_df = pd.DataFrame(titulo_tfidf.toarray(), 
#                                    columns=[f'titulo_tfidf_{i}' for i in range(titulo_tfidf.shape[1])])
#     df = pd.concat([df, titulo_tfidf_df], axis=1)

# if df['descripción'].notna().sum() > 1:
#     desc_tfidf = tfidf_desc.fit_transform(df['descripción'].fillna(''))
#     desc_tfidf_df = pd.DataFrame(desc_tfidf.toarray(), 
#                                 columns=[f'desc_tfidf_{i}' for i in range(desc_tfidf.shape[1])])
#     df = pd.concat([df, desc_tfidf_df], axis=1)

# # 3. Procesamiento de edad
# # ---------------------

# # Función para extraer edades mínimas y máximas con las nuevas reglas
# def extract_age_range(age_text):
#     if pd.isna(age_text):
#         return np.nan, np.nan
    
#     # Convertir a texto en minúsculas para facilitar los patrones
#     text = str(age_text).lower()
    
#     # Lista para almacenar todas las edades encontradas
#     ages = []
    
#     # 1. Patrón: "de X a Y años" o "entre X y Y años"
#     range_patterns = [
#         r'de (\d+) a (\d+) años',
#         r'entre (\d+) y (\d+) años',
#         r'desde (\d+) hasta (\d+) años',
#         r'desde los (\d+) años hasta los (\d+)',
#         r'desde los (\d+) hasta los (\d+) años',
#         r'(\d+) y (\d+) años'
#     ]
    
#     for pattern in range_patterns:
#         matches = re.search(pattern, text)
#         if matches:
#             min_age = int(matches.group(1))
#             max_age = int(matches.group(2))
#             ages.extend([min_age, max_age])
#             break
    
#     # 2. Patrón: "a partir de X años"
#     start_patterns = [
#         r'a partir de (\d+) años',
#         r'a partir (\d+) años',
#         r'para mayores de (\d+) años',
#         r'mayores de (\d+) años',
#         r'mayores de\s*(\d+) años'
#     ]
    
#     for pattern in start_patterns:
#         matches = re.search(pattern, text)
#         if matches and not ages:  # Solo si no encontramos un rango antes
#             min_age = int(matches.group(1))
#             max_age = 18  # Límite superior de 18 años
#             ages.extend([min_age, max_age])
#             break
    
#     # 3. Patrón: "hasta X años" o "menores de X años"
#     end_patterns = [
#         r'hasta (\d+) años',
#         r'menores de (\d+) años'
#     ]
    
#     for pattern in end_patterns:
#         matches = re.search(pattern, text)
#         if matches and not ages:  # Solo si no encontramos otro patrón antes
#             min_age = 0  # Asumimos desde 0 años
#             max_age = int(matches.group(1))
#             ages.extend([min_age, max_age])
#             break
    
#     # 4. Patrón: "X años" (edad específica)
#     single_age = re.search(r'(\d+) años', text)
#     if single_age and not ages:  # Solo si no encontramos otro patrón antes
#         age = int(single_age.group(1))
#         if age <= 18:  # Solo procesar si es menor o igual a 18
#             ages.extend([age, age])
    
#     # 5. Patrón: "de X meses" (convertir meses a años)
#     months_pattern = re.search(r'(\d+) meses', text)
#     if months_pattern and not ages:
#         months = int(months_pattern.group(1))
#         years = months / 12
#         if years <= 18:
#             ages.extend([years, years])
    
#     # 6. Patrón: "de 0 meses a X años" o variantes
#     mixed_pattern = re.search(r'de 0 meses a (\d+) años', text)
#     if mixed_pattern and not ages:
#         max_age = int(mixed_pattern.group(1))
#         if max_age <= 18:
#             ages.extend([0, max_age])
    
#     # 7. Caso especial para "nacidos entre años"
#     birth_years = re.findall(r'nacidos entre los años (\d{4}) y (\d{4})', text)
#     if birth_years and not ages:
#         current_year = datetime.now().year
#         birth_min = int(birth_years[0][0])
#         birth_max = int(birth_years[0][1])
#         age_max = current_year - birth_min
#         age_min = current_year - birth_max
#         if age_min <= 18:  # Solo consideramos si al menos parte del rango es menor o igual a 18
#             age_min = max(0, age_min)
#             age_max = min(18, age_max)
#             ages.extend([age_min, age_max])
    
    
#     # Si encontramos edades, procesarlas
#     if ages:
#         min_age = min(ages)
#         max_age = max(ages)
        
#         # Limitar edades a 18 años como máximo
#         min_age = min(min_age, 18)
#         max_age = min(max_age, 18)
        
#         return min_age, max_age
    
#     # Si no se encontró ningún patrón válido
#     return np.nan, np.nan

# # Aplicar la función y crear columnas para edad mínima y máxima
# df['edad_min'], df['edad_max'] = zip(*df['edad'].apply(extract_age_range))

# # 4. Procesamiento de fechas y horarios
# # ----------------------------------

# # Función para extraer fecha de inicio y fin
# def extract_dates(fecha_text):
#     if pd.isna(fecha_text):
#         return np.nan, np.nan
    
#     # Patrón para fechas como "Sábado 26 de abril de 2025"
#     single_date_pattern = r'(\w+) (\d+) de (\w+) de (\d{4})'
    
#     # Patrón para rangos como "Del lunes 23 de junio de 2025 al viernes 18 de julio de 2025"
#     range_date_pattern = r'Del .+ (\d+) de (\w+) de (\d{4}) al .+ (\d+) de (\w+) de (\d{4})'
    
#     # Mapeo de meses en español a números
#     month_map = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
#                 'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
#     # Intentar matchear un rango de fechas
#     range_match = re.search(range_date_pattern, fecha_text)
#     if range_match:
#         start_day, start_month_name, start_year, end_day, end_month_name, end_year = range_match.groups()
#         start_date = f"{start_year}-{month_map.get(start_month_name.lower(), 1):02d}-{int(start_day):02d}"
#         end_date = f"{end_year}-{month_map.get(end_month_name.lower(), 1):02d}-{int(end_day):02d}"
#         return start_date, end_date
    
#     # Intentar matchear una fecha única
#     single_match = re.search(single_date_pattern, fecha_text)
#     if single_match:
#         _, day, month_name, year = single_match.groups()
#         date_str = f"{year}-{month_map.get(month_name.lower(), 1):02d}-{int(day):02d}"
#         return date_str, date_str
    
#     return np.nan, np.nan

# # Aplicar la función y crear columnas para fecha inicio y fin
# df['fecha_inicio'], df['fecha_fin'] = zip(*df['fecha'].apply(extract_dates))

# # Convertir fechas a formato datetime y crear características numéricas
# for col in ['fecha_inicio', 'fecha_fin']:
#     df[col] = pd.to_datetime(df[col], errors='coerce')
#     if df[col].notna().any():
#         df[f'{col}_year'] = df[col].dt.year
#         df[f'{col}_month'] = df[col].dt.month
#         df[f'{col}_day'] = df[col].dt.day
#         df[f'{col}_dayofweek'] = df[col].dt.dayofweek

# # Calcular duración del evento en días
# df['duracion_dias'] = (df['fecha_fin'] - df['fecha_inicio']).dt.days + 1
# df['duracion_dias'] = df['duracion_dias'].fillna(1)  # Asumimos 1 día para eventos sin rango

# # Procesar horario
# def extract_hours(horario_text):
#     if pd.isna(horario_text):
#         return np.nan, np.nan
    
#     # Patrón para "a las X horas"
#     single_time_pattern = r'a las (\d+)(?::(\d+))? horas'
    
#     # Patrón para rangos como "de 16:00 a 18:00 horas"
#     range_time_pattern = r'de (\d+)(?::(\d+))? a (\d+)(?::(\d+))? horas'
    
#     # Intentar matchear un rango de horas
#     range_match = re.search(range_time_pattern, horario_text)
#     if range_match:
#         groups = range_match.groups()
#         start_hour = int(groups[0])
#         start_min = int(groups[1]) if groups[1] else 0
#         end_hour = int(groups[2])
#         end_min = int(groups[3]) if groups[3] else 0
        
#         start_time = start_hour + start_min/60
#         end_time = end_hour + end_min/60
#         duration = end_time - start_time
        
#         return start_time, duration
    
#     # Intentar matchear una hora única
#     single_match = re.search(single_time_pattern, horario_text)
#     if single_match:
#         groups = single_match.groups()
#         hour = int(groups[0])
#         minute = int(groups[1]) if groups[1] else 0
        
#         start_time = hour + minute/60
#         return start_time, 1.5  # Asumimos 1.5 horas de duración
    
#     return np.nan, np.nan

# # Aplicar la función y crear columnas para hora inicio y duración
# df['hora_inicio'], df['duracion_horas'] = zip(*df['horario'].apply(extract_hours))

# # 5. Procesamiento de días de la semana
# # ----------------------------------

# # Crear codificación one-hot para los días de la semana
# dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']

# for dia in dias_semana:
#     df[f'dia_{dia}'] = df['día_días'].str.contains(dia, case=False, na=False).astype(int)

# # 6. Procesamiento de precios
# # -----------------------

# # Codificar precios
# df['precio_gratuito'] = df['precio'].str.contains('Gratuito', case=False, na=False).astype(int)

# # 7. Procesamiento de ubicaciones
# # ---------------------------

# # Lista completa de distritos de Madrid
# distritos_madrid = [
#     'Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartín', 'Tetuán', 'Chamberí',
#     'Fuencarral-El Pardo', 'Moncloa-Aravaca', 'Latina', 'Carabanchel', 'Usera',
#     'Puente de Vallecas', 'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
#     'Villa de Vallecas', 'Vicálvaro', 'San Blas-Canillejas', 'Barajas'
# ]

# # Función para extraer distrito
# def extract_district(row):
#     # Buscar distrito en paréntesis en el nombre del lugar
#     if pd.notna(row['lugar_nombre']):
#         district_match = re.search(r'\(([^)]+)\)', row['lugar_nombre'])
#         if district_match:
#             district = district_match.group(1)
#             # Verificar si es un distrito conocido o contiene uno
#             for d in distritos_madrid:
#                 if d in district:
#                     return d
    
#     # Buscar en lugar_nombre sin paréntesis
#     if pd.notna(row['lugar_nombre']):
#         for distrito in distritos_madrid:
#             if distrito in row['lugar_nombre']:
#                 return distrito
    
#     # Buscar en título
#     if pd.notna(row['título']):
#         for distrito in distritos_madrid:
#             if distrito in row['título']:
#                 return distrito
    
#     # Buscar en descripción
#     if pd.notna(row['descripción']):
#         for distrito in distritos_madrid:
#             if distrito in row['descripción']:
#                 return distrito
    
#     # Si no se encuentra ningún distrito
#     return 'No especificado'

# # Aplicar la función para extraer distrito
# df['distrito'] = df.apply(extract_district, axis=1)

# # Convertir distrito a numérico usando one-hot encoding
# ohe = OneHotEncoder(sparse_output=False)
# distrito_encoded = ohe.fit_transform(df[['distrito']])
# distrito_df = pd.DataFrame(
#     distrito_encoded, 
#     columns=[f'distrito_{cat}' for cat in ohe.categories_[0]]
# )
# df = pd.concat([df, distrito_df], axis=1)

# # 8. Características booleanas
# # -----------------------

# # Requerimiento de inscripción
# df['requiere_inscripcion'] = df['inscripción'].notna() & ~df['inscripción'].str.contains('No especificada', na=False)
# df['requiere_inscripcion'] = df['requiere_inscripcion'].astype(int)

# # Recomendación para niños
# df['recomendado_ninos'] = df['recomendación'].str.contains('niñas y niños', case=False, na=False).astype(int)

# # 9. Características del URL
# # ----------------------

# # Comprobar si tiene URL para ampliar información
# df['tiene_url_info'] = (~df['url_ampliar_info'].isna() & ~df['url_ampliar_info'].str.contains('No disponible', na=False)).astype(int)

# # 10. Limpieza final y preparación para ML
# # ------------------------------------

# # Agregar columna ciudad_madrid con todos sus valores 1
# df['ciudad_madrid'] = 1

# # Eliminar columnas originales de texto y otras columnas innecesarias
# columnas_a_eliminar = [
#     'título', 'descripción', 'edad', 'inscripción', 'periodicidad', 'día_días', 
#     'horario', 'fecha', 'lugar_nombre', 'lugar_dirección', 'precio', 'recomendación',
#     'url_ampliar_info', 'url_actividad', 'distrito', 'fecha_inicio', 'fecha_fin'
# ]

# # Verificar qué columnas existen antes de eliminarlas
# cols_to_drop = [col for col in columnas_a_eliminar if col in df.columns]
# df_ml = df.drop(columns=cols_to_drop)

# # Manejar valores faltantes en columnas numéricas
# # for col in df_ml.select_dtypes(include=[np.number]).columns:
# #     # Usar .item() para extraer el valor escalar
# #     fill_value = 0 if df_ml[col].isna().all().item() else df_ml[col].median()
# #     df_ml[col] = df_ml[col].fillna(fill_value)

# # 11. Normalización/Estandarización
# # ----------------------------

# # Aplicar StandardScaler a las columnas numéricas
# scaler = StandardScaler()
# numeric_cols = df_ml.select_dtypes(include=[np.number]).columns
# if len(numeric_cols) > 0:
#     df_ml[numeric_cols] = scaler.fit_transform(df_ml[numeric_cols])

# # Resetear el índice una última vez
# df_ml = df_ml.reset_index(drop=True)

# # Mostrar las primeras filas del dataframe resultante
# print(df_ml.head())

# # Obtener estadísticas básicas
# print(df_ml.describe())

# # También podemos ver qué columnas tenemos ahora
# print(df_ml.columns.tolist())

In [17]:
df['edad'].unique()

array(['Talleres creativos para niños y niñas de 3 a16 años en idioma serbio',
       'de 8 a 12 años', 'de 9 a 12 años', 'de 8 a 11 años',
       'de 6 a 17 años', 'de 6 a 10 años', 'de 5 a 12 años',
       'de 12 a 17 años', 'entre 7 y 17 años', 'entre 10 y 14 años',
       'a partir de 9 años', 'entre 8 y 11 años', 'entre 9 y 12 años',
       'de 6 a 12 años', 'de 7 a 9 años', '5 y 6 años',
       'entre 4 y 10 años', 'de 6 a 8 años', nan, 'a partir de 6 años',
       'sus anécdotas y su importancia como centro cultural del Madrid de los últimos 150 años',
       'Para niñas y niños hasta 14 años',
       'Niños y niñas de 4 años a 12 años', 'a partir de 2 años',
       'a partir de 5 años', 'entre 4 y 8 años',
       '¡Únete a conocer la lengua japonesa y celebrar la lectura con nosotros!Invitamos a niños de entre 3 a 12 años a una actividad de cuentacuentos bilingüe',
       'Para peques de más de 6 años', 'a partir de 4 años',
       'a partir de 3 años', 'de diferentes ta- maños

In [25]:
# Versión 2 preparación para Machine Learning
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
from datetime import datetime
import re
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

# Hacer una copia del dataframe original para no alterarlo
df_ml = df.copy()

# 1. Limpieza básica de datos
# ------------------
df_ml.replace(["No especificada", "No especificado", "No disponible"], np.nan, inplace=True)

# 2. Procesamiento de columnas de texto con TF-IDF
# ----------------------------------
# Extraer longitud del título y descripción
df_ml['titulo_longitud'] = df_ml['título'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)
df_ml['descripcion_longitud'] = df_ml['descripción'].apply(lambda x: len(str(x)) if pd.notna(x) else 0)

# TF-IDF para título y descripción (reducido a 5 componentes para cada uno)
# TF-IDF significa Term Frequency-Inverse Document Frequency
# Convierte texto en características numéricas basadas en frecuencia de palabras
tfidf_titulo = TfidfVectorizer(max_features=5, stop_words=['de', 'la', 'el', 'en', 'y', 'a', 'para'])
tfidf_desc = TfidfVectorizer(max_features=5, stop_words=['de', 'la', 'el', 'en', 'y', 'a', 'para'])

# Solo aplicamos TF-IDF si tenemos suficientes datos no nulos
if df_ml['título'].notna().sum() > 1:
    titulo_tfidf = tfidf_titulo.fit_transform(df_ml['título'].fillna(''))
    titulo_tfidf_df = pd.DataFrame(titulo_tfidf.toarray(), 
                               columns=[f'titulo_tfidf_{i}' for i in range(titulo_tfidf.shape[1])])
    df_ml = pd.concat([df_ml, titulo_tfidf_df], axis=1)

if df_ml['descripción'].notna().sum() > 1:
    desc_tfidf = tfidf_desc.fit_transform(df_ml['descripción'].fillna(''))
    desc_tfidf_df = pd.DataFrame(desc_tfidf.toarray(), 
                            columns=[f'desc_tfidf_{i}' for i in range(desc_tfidf.shape[1])])
    df_ml = pd.concat([df_ml, desc_tfidf_df], axis=1)

# 3. Procesamiento de edad mejorado
# ---------------------
def extract_age_range(age_text):
    if pd.isna(age_text):
        return np.nan, np.nan
    
    # Convertir a texto en minúsculas para facilitar los patrones
    text = str(age_text).lower()
    
    # Lista para almacenar todas las edades encontradas
    ages = []
    
    # 1. Patrón: "de X a Y años" o "entre X y Y años"
    range_patterns = [
        r'de (\d+) a (\d+) años',
        r'entre (\d+) y (\d+) años',
        r'desde (\d+) hasta (\d+) años',
        r'desde los (\d+) años hasta los (\d+)',
        r'desde los (\d+) hasta los (\d+) años',
        r'(\d+) y (\d+) años'
    ]
    
    for pattern in range_patterns:
        matches = re.search(pattern, text)
        if matches:
            min_age = int(matches.group(1))
            max_age = int(matches.group(2))
            ages.extend([min_age, max_age])
            break
    
    # 2. Patrón: "a partir de X años"
    start_patterns = [
        r'a partir de (\d+) años',
        r'a partir (\d+) años',
        r'para mayores de (\d+) años',
        r'mayores de (\d+) años',
        r'mayores de\s*(\d+) años'
    ]
    
    for pattern in start_patterns:
        matches = re.search(pattern, text)
        if matches and not ages:  # Solo si no encontramos un rango antes
            min_age = int(matches.group(1))
            max_age = 18  # Límite superior de 18 años
            ages.extend([min_age, max_age])
            break
    
    # 3. Patrón: "hasta X años" o "menores de X años"
    end_patterns = [
        r'hasta (\d+) años',
        r'menores de (\d+) años'
    ]
    
    for pattern in end_patterns:
        matches = re.search(pattern, text)
        if matches and not ages:  # Solo si no encontramos otro patrón antes
            min_age = 0  # Asumimos desde 0 años
            max_age = int(matches.group(1))
            ages.extend([min_age, max_age])
            break
    # 4. Patrón: "X años" (edad específica)
    single_age = re.search(r'(\d+) años', text)
    if single_age and not ages:  # Solo si no encontramos otro patrón antes
        age = int(single_age.group(1))
        if age <= 18:  # Solo procesar si es menor o igual a 18
            ages.extend([age, age])
    
    # 5. Patrón: "de X meses" (convertir meses a años)
    months_pattern = re.search(r'(\d+) meses', text)
    if months_pattern and not ages:
        months = int(months_pattern.group(1))
        years = months / 12
        if years <= 18:
            ages.extend([years, years])
    
    # 6. Patrón: "de 0 meses a X años" o variantes
    mixed_pattern = re.search(r'de 0 meses a (\d+) años', text)
    if mixed_pattern and not ages:
        max_age = int(mixed_pattern.group(1))
        if max_age <= 18:
            ages.extend([0, max_age])
    
    # 7. Caso especial para "nacidos entre años"
    birth_years = re.findall(r'nacidos entre los años (\d{4}) y (\d{4})', text)
    if birth_years and not ages:
        current_year = datetime.now().year
        birth_min = int(birth_years[0][0])
        birth_max = int(birth_years[0][1])
        age_max = current_year - birth_min
        age_min = current_year - birth_max
        if age_min <= 18:  # Solo consideramos si al menos parte del rango es menor o igual a 18
            age_min = max(0, age_min)
            age_max = min(18, age_max)
            ages.extend([age_min, age_max])
       
    # Si encontramos edades, procesarlas
    if ages:
        min_age = min(ages)
        max_age = max(ages)
        
        # Limitar edades a 18 años como máximo
        min_age = min(min_age, 18)
        max_age = min(max_age, 18)
        
        return min_age, max_age
    
    # Si no se encontró ningún patrón válido
    return np.nan, np.nan

# Aplicar la función y crear columnas para edad mínima y máxima
df_ml['edad_min'], df_ml['edad_max'] = zip(*df_ml['edad'].apply(extract_age_range))

# Función para extraer fecha de inicio y fin
def extract_dates(fecha_text):
    if pd.isna(fecha_text):
        return np.nan, np.nan
    
    # Patrón para fechas como "Sábado 26 de abril de 2025"
    single_date_pattern = r'(\w+) (\d+) de (\w+) de (\d{4})'
    
    # Patrón para rangos como "Del lunes 23 de junio de 2025 al viernes 18 de julio de 2025"
    range_date_pattern = r'Del .+ (\d+) de (\w+) de (\d{4}) al .+ (\d+) de (\w+) de (\d{4})'
    
    # Mapeo de meses en español a números
    month_map = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4, 'mayo': 5, 'junio': 6,
                'julio': 7, 'agosto': 8, 'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
    # Intentar matchear un rango de fechas
    range_match = re.search(range_date_pattern, fecha_text)
    if range_match:
        start_day, start_month_name, start_year, end_day, end_month_name, end_year = range_match.groups()
        start_date = f"{start_year}-{month_map.get(start_month_name.lower(), 1):02d}-{int(start_day):02d}"
        end_date = f"{end_year}-{month_map.get(end_month_name.lower(), 1):02d}-{int(end_day):02d}"
        return start_date, end_date
    
    # Intentar matchear una fecha única
    single_match = re.search(single_date_pattern, fecha_text)
    if single_match:
        _, day, month_name, year = single_match.groups()
        date_str = f"{year}-{month_map.get(month_name.lower(), 1):02d}-{int(day):02d}"
        return date_str, date_str
    
    return np.nan, np.nan

# Aplicar la función y crear columnas para fecha inicio y fin
df['fecha_inicio'], df['fecha_fin'] = zip(*df['fecha'].apply(extract_dates))

# Convertir fechas a formato datetime y crear características numéricas
for col in ['fecha_inicio', 'fecha_fin']:
    df[col] = pd.to_datetime(df[col], errors='coerce')
    if df[col].notna().any():
        df[f'{col}_year'] = df[col].dt.year
        df[f'{col}_month'] = df[col].dt.month
        df[f'{col}_day'] = df[col].dt.day
        df[f'{col}_dayofweek'] = df[col].dt.dayofweek

# Calcular duración del evento en días
df['duracion_dias'] = (df['fecha_fin'] - df['fecha_inicio']).dt.days + 1
df['duracion_dias'] = df['duracion_dias'].fillna(1)  # Asumimos 1 día para eventos sin rango

# Procesar horario
def extract_hours(horario_text):
    if pd.isna(horario_text):
        return np.nan, np.nan
    
    # Patrón para "a las X horas"
    single_time_pattern = r'a las (\d+)(?::(\d+))? horas'
    
    # Patrón para rangos como "de 16:00 a 18:00 horas"
    range_time_pattern = r'de (\d+)(?::(\d+))? a (\d+)(?::(\d+))? horas'
    
    # Intentar matchear un rango de horas
    range_match = re.search(range_time_pattern, horario_text)
    if range_match:
        groups = range_match.groups()
        start_hour = int(groups[0])
        start_min = int(groups[1]) if groups[1] else 0
        end_hour = int(groups[2])
        end_min = int(groups[3]) if groups[3] else 0
        
        start_time = start_hour + start_min/60
        end_time = end_hour + end_min/60
        duration = end_time - start_time
        
        return start_time, duration
    
    # Intentar matchear una hora única
    single_match = re.search(single_time_pattern, horario_text)
    if single_match:
        groups = single_match.groups()
        hour = int(groups[0])
        minute = int(groups[1]) if groups[1] else 0
        
        start_time = hour + minute/60
        return start_time, 1.5  # Asumimos 1.5 horas de duración
    
    return np.nan, np.nan

# Aplicar la función y crear columnas para hora inicio y duración
df['hora_inicio'], df['duracion_horas'] = zip(*df['horario'].apply(extract_hours))

# 5. Procesamiento de días de la semana
# ----------------------------------

# Crear codificación one-hot para los días de la semana
dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']

for dia in dias_semana:
    df[f'dia_{dia}'] = df['día_días'].str.contains(dia, case=False, na=False).astype(int)

# 6. Procesamiento de precios
# -----------------------

# Codificar precios
df['precio_gratuito'] = df['precio'].str.contains('Gratuito', case=False, na=False).astype(int)

# 7. Procesamiento de ubicaciones
# ---------------------------

# Lista completa de distritos de Madrid
distritos_madrid = [
    'Centro', 'Arganzuela', 'Retiro', 'Salamanca', 'Chamartín', 'Tetuán', 'Chamberí',
    'Fuencarral-El Pardo', 'Moncloa-Aravaca', 'Latina', 'Carabanchel', 'Usera',
    'Puente de Vallecas', 'Moratalaz', 'Ciudad Lineal', 'Hortaleza', 'Villaverde',
    'Villa de Vallecas', 'Vicálvaro', 'San Blas-Canillejas', 'Barajas'
]

# Función para extraer distrito
def extract_district(row):
    # Buscar distrito en paréntesis en el nombre del lugar
    if pd.notna(row['lugar_nombre']):
        district_match = re.search(r'\(([^)]+)\)', row['lugar_nombre'])
        if district_match:
            district = district_match.group(1)
            # Verificar si es un distrito conocido o contiene uno
            for d in distritos_madrid:
                if d in district:
                    return d
    
    # Buscar en lugar_nombre sin paréntesis
    if pd.notna(row['lugar_nombre']):
        for distrito in distritos_madrid:
            if distrito in row['lugar_nombre']:
                return distrito
    
    # Buscar en título
    if pd.notna(row['título']):
        for distrito in distritos_madrid:
            if distrito in row['título']:
                return distrito
    
    # Buscar en descripción
    if pd.notna(row['descripción']):
        for distrito in distritos_madrid:
            if distrito in row['descripción']:
                return distrito
    
    # Si no se encuentra ningún distrito
    return 'No especificado'

# Aplicar la función para extraer distrito
df['distrito'] = df.apply(extract_district, axis=1)

# Convertir distrito a numérico usando one-hot encoding
ohe = OneHotEncoder(sparse_output=False)
distrito_encoded = ohe.fit_transform(df[['distrito']])
distrito_df = pd.DataFrame(
    distrito_encoded, 
    columns=[f'distrito_{cat}' for cat in ohe.categories_[0]]
)
df = pd.concat([df, distrito_df], axis=1)

# 8. Características booleanas
# -----------------------

# Requerimiento de inscripción
df['requiere_inscripcion'] = df['inscripción'].notna() & ~df['inscripción'].str.contains('No especificada', na=False)
df['requiere_inscripcion'] = df['requiere_inscripcion'].astype(int)

# Recomendación para niños
df['recomendado_ninos'] = df['recomendación'].str.contains('niñas y niños', case=False, na=False).astype(int)

# 9. Características del URL
# ----------------------

# Comprobar si tiene URL para ampliar información
df['tiene_url_info'] = (~df['url_ampliar_info'].isna() & ~df['url_ampliar_info'].str.contains('No disponible', na=False)).astype(int)

# 10. Limpieza final y preparación para ML
# ------------------------------------

# Agregar columna ciudad_madrid con todos sus valores 1
df['ciudad_madrid'] = 1


# Al final, antes de eliminar columnas, explicaremos por qué lo hacemos:

# EXPLICACIÓN: Se eliminan las columnas originales que ya han sido procesadas
# - 'día_días': Ya se ha transformado en columnas one-hot para cada día
# - Las columnas de texto: Ya se han extraído características (longitud, TF-IDF)

# Lista de columnas a eliminar (columnas originales que ya están procesadas)
columnas_a_eliminar = [
    'título', 'descripción', 'edad', 'inscripción', 'periodicidad', 'día_días', 
    'horario', 'fecha', 'lugar_nombre', 'lugar_dirección', 'precio', 'recomendación',
    'url_ampliar_info', 'url_actividad', 'distrito', 'fecha_inicio', 'fecha_fin'
]

# IMPORTANTE: NO vamos a eliminar 'distrito', 'fecha_inicio', 'fecha_fin' como en el código original
# para que puedas analizar estas columnas si lo deseas

# Verificar qué columnas existen antes de eliminarlas
cols_to_drop = [col for col in columnas_a_eliminar if col in df_ml.columns]
df_ml = df_ml.drop(columns=cols_to_drop)

# OPCIONAL: Normalización/Estandarización

# Aplicar StandardScaler a las columnas numéricas
scaler = StandardScaler()
numeric_cols = df_ml.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    df_ml[numeric_cols] = scaler.fit_transform(df_ml[numeric_cols])


print("Dataframe preparado para machine learning con forma:", df_ml.shape)

# Obtener estadísticas básicas
print(df_ml.describe())

# También podemos ver qué columnas tenemos ahora
print(df_ml.columns.tolist())

Dataframe preparado para machine learning con forma: (250, 239)
       titulo_longitud  descripcion_longitud  titulo_tfidf_0  titulo_tfidf_1  \
count     2.500000e+02          2.500000e+02    2.500000e+02    2.500000e+02   
mean      2.842171e-17          6.750156e-17    5.684342e-17    7.105427e-18   
std       1.002006e+00          1.002006e+00    1.002006e+00    1.002006e+00   
min      -1.521455e+00         -8.554341e-01   -2.888565e-01   -2.670255e-01   
25%      -7.483578e-01         -6.911974e-01   -2.888565e-01   -2.670255e-01   
50%      -2.513669e-01         -4.709710e-01   -2.888565e-01   -2.670255e-01   
75%       6.321724e-01          3.390143e-01   -2.888565e-01   -2.670255e-01   
max       5.049869e+00          4.571841e+00    4.825953e+00    3.975518e+00   

       titulo_tfidf_2  titulo_tfidf_3  titulo_tfidf_4  desc_tfidf_0  \
count    2.500000e+02      250.000000    2.500000e+02  2.500000e+02   
mean    -2.842171e-17        0.000000   -2.131628e-17 -3.552714e-18   
st

In [26]:
df_ml

,titulo_longitud,descripcion_longitud,titulo_tfidf_0,titulo_tfidf_1,titulo_tfidf_2,titulo_tfidf_3,titulo_tfidf_4,desc_tfidf_0,desc_tfidf_1,desc_tfidf_2,...,titulo_tfidf_0,titulo_tfidf_1,titulo_tfidf_2,titulo_tfidf_3,titulo_tfidf_4,desc_tfidf_0,desc_tfidf_1,desc_tfidf_2,desc_tfidf_3,desc_tfidf_4
0,0.190403,0.316618,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,-0.437610,-0.540783,-0.411263,...,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,-0.437610,-0.540783,-0.411263,-0.367248,-0.338258
1,1.018721,-0.743455,3.419541,-0.267026,2.922575,-0.276010,-0.271957,-0.437610,-0.540783,-0.411263,...,3.419541,-0.267026,2.922575,-0.276010,-0.271957,-0.437610,-0.540783,-0.411263,-0.367248,-0.338258
2,1.018721,2.645793,3.419541,-0.267026,2.922575,-0.276010,-0.271957,0.340413,1.001578,-0.411263,...,3.419541,-0.267026,2.922575,-0.276010,-0.271957,0.340413,1.001578,-0.411263,3.080831,2.105260
3,1.515712,0.309153,2.683733,-0.267026,2.279921,-0.276010,2.833399,-0.437610,-0.540783,-0.411263,...,2.683733,-0.267026,2.279921,-0.276010,2.833399,-0.437610,-0.540783,-0.411263,-0.367248,-0.338258
4,0.908278,2.645793,3.419541,-0.267026,2.922575,-0.276010,-0.271957,0.340413,1.001578,-0.411263,...,3.419541,-0.267026,2.922575,-0.276010,-0.271957,0.340413,1.001578,-0.411263,3.080831,2.105260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1.073942,-0.116369,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,3.159608,-0.540783,-0.411263,...,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,3.159608,-0.540783,-0.411263,-0.367248,-0.338258
246,0.908278,1.570789,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,2.055428,1.930328,-0.411263,...,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,2.055428,1.930328,-0.411263,1.474212,-0.338258
247,1.681375,1.921658,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,1.867643,0.982535,1.166906,...,-0.288857,-0.267026,-0.316341,-0.276010,-0.271957,1.867643,0.982535,1.166906,1.903090,-0.338258
248,0.135182,-0.855434,-0.288857,-0.267026,-0.316341,3.776902,-0.271957,-0.437610,-0.540783,-0.411263,...,-0.288857,-0.267026,-0.316341,3.776902,-0.271957,-0.437610,-0.540783,-0.411263,-0.367248,-0.338258


______________________________________

Al final, guardamos el dataframe obtenido en processed.

In [20]:
# Guardar el DataFrame modificado (sin columna fechas)

# Definir ruta del archivo a guardar:
directorio = '../data/processed/'
nombre_archivo_procesado = 'actividades(250)_detalles_24-04-2025.csv'
ruta_archivo_trabajo = os.path.join(directorio, nombre_archivo_procesado)

df.to_csv(ruta_archivo_trabajo, index=False, encoding='utf-8-sig')
print(f"Datos procesados guardados en carpeta '{ruta_archivo_trabajo}'")

Datos procesados guardados en carpeta '../data/processed/actividades(250)_detalles_24-04-2025.csv'
